# Google Earth Engine Variable Extraction Notebook

This notebook provides functions to extract various environmental variables from Google Earth Engine (GEE) datasets.
The primary goals are:
- Define functions to retrieve specific variables based on user-defined parameters (region, time period, frequency).
- For time-series data, aggregate it to hourly, daily, monthly, or yearly means.
- Save the extracted data for each variable into a separate CSV file.

**Instructions:**
1. Run the GEE Authentication and Initialization cell first. You will need to authenticate with a Google account that has GEE access.
2. Define your region of interest (AOI) as a GeoJSON-like dictionary.
3. Call the specific extraction functions for the variables you need, providing the AOI, date range, frequency, and output directory.

In [14]:
# CONFIG VALUES
# BELA BELA -24.868138, 28.374596
# Zona sin vegetacion '''24°52'48"S 28°24'43"E'''  '''24°52'37"S 28°24'07"E'''
# Zona con vegetacion '''24°52'12"S 28°22'26"E'''

# AlHenakiyah (SaudiArabia)
# 24.52594, 40.74656

# Phofu sitio (poca vegetacion, sin arboles)
# 27°04'47"S 26°52'12"E
# Vierfontein, South Africa
# '''-27.0844° 26.8722'''
# '''27°05'01"S 26°52'18"E'''

# Onderstepoort Cluster Site
# '''25°26'05"S 27°01'07"E'''

# El Sanate (Guatemala)
# '''14.02 N -90.37 W'''

# Valle 1-4
# ''' 41°41'49"N 4°54'30"W'''



# Location
LOCATION_NAME = 'Valle 3&4 (Valladolid, Spain)'
LATITUDE = '''27°05'01"S'''
LONGITUDE =  '''26°52'18"E'''
DESCRIPCION = 'Albedo 5km radio'

# Area of interest
POLIGON_VERTICES = 6
RADIO = 5 # Kilometers

# PERIOD
INITIAL_DATE = '2001-01-01'
FINAL_DATE = '2023-11-30'
# Define the desired frequency for time-series aggregation in csv file
# Options: 'hourly', 'daily', 'monthly', 'yearly'
FRECUENCY = 'daily'



# GEE project
PROJECT='gen-lang-client-0253961861'

In [2]:
# Run this once per session
# Update folium to latest version

!sudo apt-get install wkhtmltopdf
!pip install folium html2image
!pip install --upgrade folium
!pip install playwright
!playwright install
!pip install reportlab

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libudev1 libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 

In [3]:
import ee
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import os
import csv
import math # Ensure math is imported for potential calculations like wind
import calendar

In [4]:
# Trigger the authentication flow.
# This will print a URL, open it, authorize, and copy the code back into the input box.
try:
    ee.Authenticate()
except Exception as e:
    print(f"Authentication failed or already authenticated: {e}")
    # For automated environments or if auth was done in a previous session,
    # this might raise an error if not needed, so we can often proceed.

# Initialize the library.
# Replace 'YOUR_GEE_PROJECT' with your actual GEE project ID if you have one,
# otherwise, GEE often can use a default cloud project associated with your account.
try:
    ee.Initialize(project='gen-lang-client-0253961861')
    print("GEE Initialized successfully.")
except Exception as e:
    try:
        # Fallback if project-specific initialization fails
        ee.Initialize()
        print("GEE Initialized successfully (default project).")
    except Exception as e_init:
        print(f"GEE Initialization failed: {e_init}")
        print("Please ensure you have authenticated and have a GEE-enabled project.")

# Define a helper to check GEE initialization status
def check_gee_initialized():
    try:
        ee.ImageCollection('MODIS/061/MCD43A3').limit(1).size().getInfo()
        print("GEE is initialized and accessible.")
        return True
    except Exception as e:
        print(f"GEE not properly initialized or accessible: {e}")
        return False

check_gee_initialized()

GEE Initialized successfully.
GEE is initialized and accessible.


True

In [5]:
# Generic Time Series Extraction Function
def extract_gee_time_series(
    variable_name: str,
    region_geojson: dict, # GeoJSON dictionary for the region
    start_date_str: str,
    end_date_str: str,
    frequency: str,  # 'hourly', 'daily', 'monthly', 'yearly'
    gee_dataset_id: str,
    gee_band_name: str, # Can be a list for multi-band calculations (e.g. wind)
    scale: int,
    output_dir: str,
    gee_project: str = None, # Optional: GEE project for initialization if needed
    reducer: ee.Reducer = ee.Reducer.mean(),
    data_scaling_factor: float = None,
    data_offset_factor: float = None,
    post_process_function: callable = None, # Function to apply to the reduced value or dictionary of values
    nan_value = None # Value to use if GEE returns None
) -> str:
    """
    Extracts time-series data from Google Earth Engine for a specified variable and frequency,
    saves it to a CSV file, and returns the path to the CSV.

    Parameters:
    - variable_name: Name of the variable (used for CSV filename).
    - region_geojson: GeoJSON dictionary defining the region of interest.
    - start_date_str: Start date in 'YYYY-MM-DD' format.
    - end_date_str: End date in 'YYYY-MM-DD' format.
    - frequency: Aggregation frequency ('hourly', 'daily', 'monthly', 'yearly').
    - gee_dataset_id: Earth Engine ImageCollection ID.
    - gee_band_name: Name of the band(s) to extract. If a list, post_process_function must handle it.
    - scale: Spatial resolution in meters for reduction.
    - output_dir: Directory to save the output CSV file.
    - gee_project: Optional GEE project ID for ee.Initialize().
    - reducer: Earth Engine reducer to apply (default: ee.Reducer.mean()).
    - data_scaling_factor: Optional factor to multiply the band data by.
    - data_offset_factor: Optional offset to add to the band data.
    - post_process_function: Optional function to apply to the raw reduced value(s).
                             It should accept a dictionary of band values if multiple bands are processed,
                             or a single value if one band is processed. It should return a dictionary
                             of processed values or a single processed value.
    - nan_value: Value to fill in if GEE returns no data for a period.

    Returns:
    - Path to the generated CSV file.
    """
    if gee_project:
        try:
            ee.Initialize(project=gee_project)
        except Exception:
            ee.Initialize() # Fallback

    ee_region = ee.Geometry(region_geojson)

    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

    os.makedirs(output_dir, exist_ok=True)

    date_periods = []
    current_date = start_date

    if frequency == 'hourly':
        delta = relativedelta(hours=1)
        date_format_label = '%Y-%m-%d %H:%M:%S'
        while current_date <= end_date:
            period_end_date = current_date + delta - relativedelta(seconds=1) # End of the hour
            date_periods.append({
                "start": current_date.strftime('%Y-%m-%dT%H:%M:%S'),
                "end": period_end_date.strftime('%Y-%m-%dT%H:%M:%S'),
                "label": current_date.strftime(date_format_label)
            })
            current_date += delta
    elif frequency == 'daily':
        delta = relativedelta(days=1)
        date_format_label = '%Y-%m-%d'
        while current_date <= end_date:
            date_periods.append({
                "start": current_date.strftime('%Y-%m-%d'),
                "end": (current_date + delta - relativedelta(days=1)).strftime('%Y-%m-%d'), # inclusive end
                "label": current_date.strftime(date_format_label)
            })
            current_date += delta
    elif frequency == 'monthly':
        delta = relativedelta(months=1)
        date_format_label = '%Y-%m'
        while current_date <= end_date:
            month_start = current_date.replace(day=1)
            month_end = month_start + delta - relativedelta(days=1)
            date_periods.append({
                "start": month_start.strftime('%Y-%m-%d'),
                "end": month_end.strftime('%Y-%m-%d'),
                "label": month_start.strftime(date_format_label)
            })
            current_date += delta
    elif frequency == 'yearly':
        delta = relativedelta(years=1)
        date_format_label = '%Y'
        while current_date <= end_date:
            year_start = current_date.replace(month=1, day=1)
            year_end = year_start + delta - relativedelta(days=1)
            date_periods.append({
                "start": year_start.strftime('%Y-%m-%d'),
                "end": year_end.strftime('%Y-%m-%d'),
                "label": year_start.strftime(date_format_label)
            })
            current_date += delta
    else:
        raise ValueError("Invalid frequency. Choose from 'hourly', 'daily', 'monthly', 'yearly'.")

    results = []

    for period in date_periods:
        print(f"Processing period {period['label']}")
        try:
            collection = ee.ImageCollection(gee_dataset_id) \
                           .filterDate(ee.Date(period["start"]), ee.Date(period["end"]).advance(1, 'day')) # GEE end date is exclusive

            if isinstance(gee_band_name, list): # For multi-band variables like wind
                selected_bands_collection = collection.select(gee_band_name)
            else: # Single band
                selected_bands_collection = collection.select([gee_band_name])

            # Check if collection is empty for the period
            if selected_bands_collection.size().getInfo() == 0:
                print(f"No images found for {variable_name} in period {period['label']} for dataset {gee_dataset_id}")
                reduced_value = nan_value
                if isinstance(gee_band_name, list) and nan_value is not None:
                     # If multiple bands expected, fill with nan_value for each
                    reduced_value = {band: nan_value for band in gee_band_name}

                if post_process_function and reduced_value is not None : # nan_value can be processed if needed
                     processed_value = post_process_function(reduced_value)
                else:
                     processed_value = reduced_value

                # Ensure processed_value is a dictionary for DataFrame creation
                if not isinstance(processed_value, dict) and isinstance(gee_band_name, list):
                    # if single value came from post_process for multiple bands, try to map it
                    # this might need adjustment based on post_process_function's behavior
                    processed_value = {f"{variable_name}_{b}" if len(gee_band_name) > 1 else variable_name : processed_value for b in gee_band_name}
                elif not isinstance(processed_value, dict):
                    processed_value = {variable_name: processed_value}

            else:
                image_for_period = selected_bands_collection.mean() # Temporal aggregation

                if data_scaling_factor is not None:
                    image_for_period = image_for_period.multiply(data_scaling_factor)
                if data_offset_factor is not None:
                    image_for_period = image_for_period.add(data_offset_factor)

                # Perform reduction
                reduction = image_for_period.reduceRegion(
                    reducer=reducer,
                    geometry=ee_region,
                    scale=scale,
                    maxPixels=1e10,
                    bestEffort=True, # Added bestEffort
                    tileScale=0.1 # Added tileScale
                )

                raw_reduced_value = {}
                if isinstance(gee_band_name, list):
                    for band in gee_band_name:
                        raw_reduced_value[band] = reduction.get(band).getInfo()
                else:
                    raw_reduced_value = reduction.get(gee_band_name).getInfo()

                if post_process_function:
                    processed_value = post_process_function(raw_reduced_value)
                else:
                    processed_value = raw_reduced_value

            # Structure for DataFrame
            current_row = {'timestamp': period['label']}
            if isinstance(processed_value, dict):
                current_row.update(processed_value)
            else: # Single value result
                current_row[variable_name] = processed_value
            results.append(current_row)

        except Exception as e:
            print(f"Error processing period {period['label']} for {variable_name}: {e}")
            # Add a row with nan_value or error indication
            error_entry = {'timestamp': period['label']}
            if isinstance(gee_band_name, list):
                for band in gee_band_name:
                    error_entry[f"{variable_name}_{band}"] = nan_value  # Or some error string
            else:
                error_entry[variable_name] = nan_value # Or some error string
            results.append(error_entry)
            continue # Continue to next period

    if not results:
        print(f"No data extracted for {variable_name} in the given period.")
        return None

    df = pd.DataFrame(results)
    # Ensure timestamp is the first column
    cols = ['timestamp'] + [col for col in df.columns if col != 'timestamp']
    df = df[cols]

    # Sanitize filename
    safe_start_date = start_date_str.replace('-', '')
    safe_end_date = end_date_str.replace('-', '')
    csv_filename = f"{variable_name}_{frequency}_{safe_start_date}_{safe_end_date}.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    df.to_csv(csv_path, index=False, na_rep=str(nan_value) if nan_value is not None else 'NaN') # Use nan_value for na_rep

    print(f"Successfully saved data for {variable_name} to {csv_path}")
    return csv_path



def extract_gee_time_series_multi(
    variable_names: list,
    region_geojson: dict,
    start_date_str: str,
    end_date_str: str,
    frequency: str,
    gee_dataset_id: str,
    gee_band_names: list,
    scale: int,
    output_dir: str,
    post_process_function: callable = None,
    nan_value: float = np.nan,
    gee_project: str = None,
    reducer: ee.Reducer = ee.Reducer.mean(),
    data_scaling_factors: dict = None,
    data_offset_factors: dict = None
) -> str:
    """
    Extracts time-series data for multiple bands from Google Earth Engine.

    Parameters:
    - variable_names: List of output variable names (e.g., ['GHI', 'DHI'])
    - region_geojson: GeoJSON dictionary defining the region of interest
    - start_date_str: Start date in 'YYYY-MM-DD' format
    - end_date_str: End date in 'YYYY-MM-DD' format
    - frequency: Aggregation frequency ('hourly', 'daily', 'monthly', 'yearly')
    - gee_dataset_id: Earth Engine ImageCollection ID
    - gee_band_names: List of band names to extract
    - scale: Spatial resolution in meters for reduction
    - output_dir: Directory to save the output CSV file
    - post_process_function: Function to process raw reduced values
    - nan_value: Value to use if GEE returns None
    - gee_project: Optional GEE project ID for ee.Initialize()
    - reducer: Earth Engine reducer to apply
    - data_scaling_factors: Dict of scaling factors per band {band: factor}
    - data_offset_factors: Dict of offset factors per band {band: offset}

    Returns:
    - Path to the generated CSV file
    """
    # Initialize Earth Engine
    if gee_project:
        try:
            ee.Initialize(project=gee_project)
        except Exception:
            ee.Initialize()
    else:
        ee.Initialize()

    # Validate input lengths
    if len(variable_names) != len(gee_band_names):
        raise ValueError("variable_names and gee_band_names must have the same length")

    # Create region and date objects
    ee_region = ee.Geometry(region_geojson)
    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Generate time periods based on frequency
    date_periods = []
    current_date = start_date

    if frequency == 'hourly':
        delta = relativedelta(hours=1)
        date_format_label = '%Y-%m-%d %H:%M:%S'
        while current_date <= end_date:
            period_end_date = current_date + delta - relativedelta(seconds=1)
            date_periods.append({
                "start": current_date.strftime('%Y-%m-%dT%H:%M:%S'),
                "end": period_end_date.strftime('%Y-%m-%dT%H:%M:%S'),
                "label": current_date.strftime(date_format_label)
            })
            current_date += delta

    elif frequency == 'daily':
        delta = relativedelta(days=1)
        date_format_label = '%Y-%m-%d'
        while current_date <= end_date:
            period_end_date = current_date + delta - relativedelta(seconds=1)
            date_periods.append({
                "start": current_date.strftime('%Y-%m-%d'),
                "end": period_end_date.strftime('%Y-%m-%d'),
                "label": current_date.strftime(date_format_label)
            })
            current_date += delta

    elif frequency == 'monthly':
        delta = relativedelta(months=1)
        date_format_label = '%Y-%m'
        while current_date <= end_date:
            month_start = current_date.replace(day=1)
            month_end = month_start + delta - relativedelta(days=1)
            date_periods.append({
                "start": month_start.strftime('%Y-%m-%d'),
                "end": month_end.strftime('%Y-%m-%d'),
                "label": month_start.strftime(date_format_label)
            })
            current_date += delta

    elif frequency == 'yearly':
        delta = relativedelta(years=1)
        date_format_label = '%Y'
        while current_date <= end_date:
            year_start = current_date.replace(month=1, day=1)
            year_end = year_start + delta - relativedelta(days=1)
            date_periods.append({
                "start": year_start.strftime('%Y-%m-%d'),
                "end": year_end.strftime('%Y-%m-%d'),
                "label": year_start.strftime(date_format_label)
            })
            current_date += delta

    else:
        raise ValueError("Invalid frequency. Choose from 'hourly', 'daily', 'monthly', 'yearly'.")

    # Prepare results storage
    results = []

    for period in date_periods:
        print(f"Processing period {period['label']}")
        try:
            # Filter image collection by date
            collection = ee.ImageCollection(gee_dataset_id) \
                           .filterDate(ee.Date(period["start"]),
                                       ee.Date(period["end"]).advance(1, 'day'))

            # Check if collection is empty
            if collection.size().getInfo() == 0:
                print(f"No images found for period {period['label']}")
                raw_values = {band: nan_value for band in gee_band_names}
            else:
                # Calculate temporal mean
                image = collection.mean()

                # Apply scaling and offset per band if provided
                if data_scaling_factors or data_offset_factors:
                    scaled_bands = []
                    for band in gee_band_names:
                        band_img = image.select(band)

                        if data_scaling_factors and band in data_scaling_factors:
                            band_img = band_img.multiply(data_scaling_factors[band])

                        if data_offset_factors and band in data_offset_factors:
                            band_img = band_img.add(data_offset_factors[band])

                        scaled_bands.append(band_img)

                    image = ee.Image.cat(*scaled_bands).rename(gee_band_names)

                # Perform reduction
                reduction = image.reduceRegion(
                    reducer=reducer,
                    geometry=ee_region,
                    scale=scale,
                    maxPixels=1e10,
                    bestEffort=True,
                    tileScale=8
                )

                # Get reduced values
                raw_values = {}
                for band in gee_band_names:
                    value = reduction.get(band).getInfo()
                    raw_values[band] = value if value is not None else nan_value

        except Exception as e:
            print(f"Error processing period {period['label']}: {e}")
            raw_values = {band: nan_value for band in gee_band_names}

        # Apply post-processing if provided
        if post_process_function:
            processed_values = post_process_function(raw_values)

            # Validate post-processing output
            if not isinstance(processed_values, dict):
                raise TypeError("post_process_function must return a dictionary")

            if set(processed_values.keys()) != set(variable_names):
                raise ValueError("post_process_function keys must match variable_names")
        else:
            processed_values = raw_values

        # Create result row
        row = {'timestamp': period['label']}
        for var_name in variable_names:
            row[var_name] = processed_values[var_name]

        results.append(row)

    # Create and save DataFrame
    if not results:
        print("No data extracted for the given period.")
        return None

    df = pd.DataFrame(results)

    # Generate filename
    safe_start = start_date_str.replace('-', '')
    safe_end = end_date_str.replace('-', '')
    csv_filename = f"{'_'.join(variable_names)}_{frequency}_{safe_start}_{safe_end}.csv"
    csv_path = os.path.join(output_dir, csv_filename)

    df.to_csv(csv_path, index=False)
    print(f"Data saved to {csv_path}")

    return csv_path

# --- Helper function for MODIS LST to Celsius ---
def convert_modis_lst_to_celsius(lst_value):
    """Converts MODIS LST (scaled Kelvin) to Celsius."""
    if lst_value is None:
        return None
    return (lst_value * 0.02) - 273.15

# --- Specific Variable Extraction Functions ---

# Albedo
def extract_albedo_bsa(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=500):
    return extract_gee_time_series(
        variable_name="Albedo_BSA",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MCD43A3', # As per reference notebook
        gee_band_name='Albedo_BSA_shortwave', # As per reference notebook
        scale=scale,
        output_dir=output_dir,
        data_scaling_factor=0.001, # MODIS albedo scaling
        nan_value=np.nan # Use numpy's NaN for missing values
    )

def extract_albedo_wsa(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=500):
    return extract_gee_time_series(
        variable_name="Albedo_WSA",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MCD43A3', # As per reference notebook
        gee_band_name='Albedo_WSA_shortwave', # As per reference notebook
        scale=scale,
        output_dir=output_dir,
        data_scaling_factor=0.001, # MODIS albedo scaling
        nan_value=np.nan
    )

# Solar Radiation (using LST Day as proxy, similar to reference notebook)
# Note: This is an approximation. A direct solar radiation dataset would be better if available and suitable.
def post_process_solar_radiation(lst_day_value):
    """Approximates solar radiation from MODIS LST Day value."""
    if lst_day_value is None:
        return None
    # If input is a dict (from multi-band processing in generic func),
    # extract the value. This handles cases where post_process_function
    # might receive a dict even for single-band selection if the generic
    # function's internal logic changes or if it's called directly with a dict.
    if isinstance(lst_day_value, dict):
        val = lst_day_value.get('LST_Day_1km', None)
        if val is None: return None
    else:
        val = lst_day_value

    temp_kelvin = val * 0.02 # MODIS LST scaling to Kelvin
    stefan_boltzmann = 5.67e-8  # W/(m^2 K^4)
    # This is a simplified Stefan-Boltzmann law application, assuming emissivity = 1
    # The reference notebook had this calculation. True GHI/DNI would come from datasets like ERA5 or specific solar datasets.
    return stefan_boltzmann * (temp_kelvin ** 4)

def extract_radiacion_solar(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=1000):
    return extract_gee_time_series(
        variable_name="Radiacion_Solar_Approximated_from_LST",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MOD11A1', # Using LST dataset as per reference
        gee_band_name='LST_Day_1km',       # Using LST Day band
        scale=scale,
        output_dir=output_dir,
        post_process_function=post_process_solar_radiation,
        nan_value=np.nan
    )

# Temperature
def extract_temperatura_dia(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=1000):
    return extract_gee_time_series(
        variable_name="Temperatura_Dia_Celsius",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MOD11A1', # As per reference notebook
        gee_band_name='LST_Day_1km',
        scale=scale,
        output_dir=output_dir,
        post_process_function=lambda x: convert_modis_lst_to_celsius(x.get('LST_Day_1km') if isinstance(x, dict) else x) if x is not None else None, # LST to Celsius
        nan_value=np.nan
    )

def extract_temperatura_noche(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=1000):
    return extract_gee_time_series(
        variable_name="Temperatura_Noche_Celsius",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MOD11A1', # As per reference notebook
        gee_band_name='LST_Night_1km',
        scale=scale,
        output_dir=output_dir,
        post_process_function=lambda x: convert_modis_lst_to_celsius(x.get('LST_Night_1km') if isinstance(x, dict) else x) if x is not None else None, # LST to Celsius
        nan_value=np.nan
    )

In [6]:
# --- Wind Speed and Direction ---
def post_process_wind_data(wind_components):
    """Calculates wind speed and direction from u and v components."""
    u = wind_components.get('u_component_of_wind_10m')
    v = wind_components.get('v_component_of_wind_10m')

    if u is None or v is None:
        return {'Viento_Velocidad': np.nan, 'Viento_Direccion': np.nan}

    speed = math.sqrt(u**2 + v**2)
    # Wind direction: meteorological convention (degrees from North, clockwise)
    # atan2(u,v) gives angle w.r.t positive y-axis (North). Convert to degrees.
    # Then adjust to be 0-360.
    # direction_rad = math.atan2(u, v) # u is eastward, v is northward
    # direction_deg = math.degrees(direction_rad)
    # direction_met = (270 - direction_deg) % 360 # As in reference notebook
    # A common formula for meteorological wind direction from u,v components:
    direction_met = (180 / math.pi) * math.atan2(-u, -v) + 180
    direction_met = direction_met % 360 # Ensure it's within 0-360

    return {'Viento_Velocidad': speed, 'Viento_Direccion': direction_met}

def extract_viento(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=10000):
    # Note: ERA5 Land is hourly. If 'daily', 'monthly', 'yearly' frequency is requested,
    # the generic function will average the hourly u/v components first, then calculate speed/direction.
    return extract_gee_time_series(
        variable_name="Viento", # Base name, will be expanded by post_process_wind_data keys
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='ECMWF/ERA5_LAND/HOURLY', # As per reference notebook
        gee_band_name=['u_component_of_wind_10m', 'v_component_of_wind_10m'],
        scale=scale,
        output_dir=output_dir,
        post_process_function=post_process_wind_data,
        nan_value=np.nan
    )

# --- Topography (Elevation, Slope, Aspect) ---
# These are static, so they don't depend on date range or frequency in the same way.
# The function will calculate mean values over the region for a single point in time (the SRTM image is static).
def extract_topography(region_geojson, output_dir, scale=30, variable_name_prefix="Topografia_"):
    """
    Extracts mean Elevation, Slope, and Aspect for a region and saves to a CSV.
    Output CSV will have one row with columns: 'Elevation', 'Slope', 'Aspect'.
    """
    ee_region = ee.Geometry(region_geojson)
    os.makedirs(output_dir, exist_ok=True)

    try:
        srtm = ee.Image('USGS/SRTMGL1_003') # SRTM is a single image
        elevation = srtm.select('elevation')
        slope = ee.Terrain.slope(elevation)
        aspect = ee.Terrain.aspect(elevation)

        topography_image = ee.Image.cat([elevation, slope, aspect]).rename(['Elevacion', 'Pendiente', 'Aspecto'])

        reduction = topography_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10,
            bestEffort=True,
            tileScale=0.1
        )

        # GetInfo once
        reduced_data = reduction.getInfo()

        # Handle cases where a key might be missing (though unlikely for these specific bands from SRTM)
        data_for_df = {
            'Elevacion': reduced_data.get('Elevacion', np.nan),
            'Pendiente': reduced_data.get('Pendiente', np.nan),
            'Aspecto': reduced_data.get('Aspecto', np.nan)
        }

        df = pd.DataFrame([data_for_df])

        # Define a simple, non-temporal filename
        csv_filename = f"{variable_name_prefix}mean_values.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))

        print(f"Successfully saved topography data to {csv_path}")
        return csv_path

    except Exception as e:
        print(f"Error extracting topography data: {e}")
        # Create a CSV with NaNs if there's an error
        df = pd.DataFrame([{'Elevacion': np.nan, 'Pendiente': np.nan, 'Aspecto': np.nan}])
        csv_filename = f"{variable_name_prefix}mean_values_error.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))
        return csv_path


# --- Land Cover (Principal Type and Percentage) ---
# This is typically analyzed for a specific year or period, not as a continuous time series like temperature.
# The function will find the dominant land cover type and its percentage for a given year.
def extract_land_cover(region_geojson, year_str, output_dir, scale=500, variable_name_prefix="Cobertura_"):
    """
    Extracts the principal land cover type and its percentage for a region and year.
    Saves to a CSV with columns: 'Year', 'Cobertura_Terrestre_Principal', 'Cobertura_Terrestre_Porcentaje'.
    Uses MODIS MCD12Q1 dataset.
    """
    ee_region = ee.Geometry(region_geojson)
    os.makedirs(output_dir, exist_ok=True)

    try:
        # MODIS Land Cover Type 1 (IGBP classification)
        # MCD12Q1 provides yearly data. Filter for the specific year.
        start_date = f"{year_str}-01-01"
        end_date = f"{year_str}-12-31" # End of the year

        land_cover_collection = ee.ImageCollection('MODIS/061/MCD12Q1') \
                                  .filterDate(start_date, end_date) \
                                  .select('LC_Type1') # IGBP classification band

        # Get the image for the year (should be one)
        land_cover_image = land_cover_collection.first()

        # A robust check to see if the image is valid and has bands
        # Attempt to get band names; if it fails or is empty, image is likely invalid/empty
        valid_image = False
        try:
            if land_cover_image.bandNames().size().getInfo() > 0:
                valid_image = True
        except Exception: # Handles cases where land_cover_image might be null or not a proper image
            valid_image = False

        if not valid_image:
             print(f"No MODIS land cover data found or image is invalid for year {year_str}.")
             data_for_df = {'Year': year_str, 'Cobertura_Terrestre_Principal': np.nan, 'Cobertura_Terrestre_Porcentaje': np.nan}
        else:
            # Calculate frequency histogram of land cover types in the region
            histogram = land_cover_image.reduceRegion(
                reducer=ee.Reducer.frequencyHistogram(),
                geometry=ee_region,
                scale=scale,
                maxPixels=1e10,
                bestEffort=True,
                tileScale=0.1
            ).get('LC_Type1') # Get the histogram for the band

            histogram_info = histogram.getInfo() # This can be slow for very large regions

            if not histogram_info: # Check if histogram is empty
                print(f"Land cover histogram is empty for year {year_str} in the region.")
                data_for_df = {'Year': year_str, 'Cobertura_Terrestre_Principal': np.nan, 'Cobertura_Terrestre_Porcentaje': np.nan}
            else:
                # Convert keys (class IDs) to integers and find the principal class
                class_counts = {int(k): v for k, v in histogram_info.items()}
                total_pixels = sum(class_counts.values())

                if total_pixels == 0:
                    principal_class_id = np.nan
                    percentage = np.nan
                else:
                    principal_class_id = max(class_counts, key=class_counts.get)
                    percentage = (class_counts[principal_class_id] / total_pixels) * 100

                data_for_df = {
                    'Year': year_str,
                    'Cobertura_Terrestre_Principal': principal_class_id,
                    'Cobertura_Terrestre_Porcentaje': percentage
                }

        df = pd.DataFrame([data_for_df])
        csv_filename = f"{variable_name_prefix}{year_str}.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))

        print(f"Successfully saved land cover data for {year_str} to {csv_path}")
        return csv_path

    except Exception as e:
        print(f"Error extracting land cover data for {year_str}: {e}")
        df = pd.DataFrame([{'Year': year_str, 'Cobertura_Terrestre_Principal': np.nan, 'Cobertura_Terrestre_Porcentaje': np.nan}])
        csv_filename = f"{variable_name_prefix}{year_str}_error.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))
        return csv_path


---
## Example Usage of Extraction Functions

Below are examples of how to use the implemented functions.
- You'll need to define your `region_of_interest` (as a GeoJSON-like Python dictionary).
- Specify your desired `start_date`, `end_date`, `year_for_landcover`, and `output_directory`.
- Uncomment the function calls you wish to run.
- Ensure the `output_directory` exists or the functions will create it.

In [7]:
import math
import re

def dms_to_decimal(coord_str):
    # First, try to parse as a decimal degree with optional direction
    decimal_pattern = re.compile(
        r'^\s*([+-]?[\d.]+)\s*([NSEW]?)\s*$',
        re.IGNORECASE
    )
    decimal_match = decimal_pattern.match(coord_str.strip())
    if decimal_match:
        number_str, direction = decimal_match.groups()
        try:
            number = float(number_str)
        except ValueError:
            pass  # Not a valid decimal, proceed to check DMS
        else:
            if direction:
                direction = direction.upper()
                decimal = abs(number)
                if direction in ['S', 'W']:
                    decimal *= -1
                return decimal
            else:
                return number

    # If not a decimal, attempt to parse as DMS
    dms_pattern = re.compile(
        r'''\s*([+-]?\d+)\s*°\s*([+-]?\d+)\s*'\s*([+-]?\d+\.?\d*)\s*"*\s*([NSEW]?)\s*''',
        re.IGNORECASE
    )
    dms_match = dms_pattern.match(coord_str.strip())
    if not dms_match:
        raise ValueError(f"Invalid coordinate format: {coord_str}")

    degrees, minutes, seconds, direction = dms_match.groups()
    degrees = float(degrees)
    minutes = float(minutes)
    seconds = float(seconds)
    decimal = degrees + minutes / 60 + seconds / 3600

    if direction.upper() in ['S', 'W']:
        decimal *= -1

    return decimal

def get_point(center_lat, center_lon, distance_km, bearing_deg):
    R = 6371  # Earth radius in kilometers
    delta = distance_km / R  # Angular distance in radians

    lat1 = math.radians(center_lat)
    lon1 = math.radians(center_lon)
    theta = math.radians(bearing_deg)

    lat2 = math.asin(
        math.sin(lat1) * math.cos(delta) +
        math.cos(lat1) * math.sin(delta) * math.cos(theta))
    lon2 = lon1 + math.atan2(
        math.sin(theta) * math.sin(delta) * math.cos(lat1),
        math.cos(delta) - math.sin(lat1) * math.sin(lat2))

    lon2 = (lon2 + 3 * math.pi) % (2 * math.pi) - math.pi

    return (math.degrees(lat2), math.degrees(lon2))

def generate_polygon_vertices(lat_input, lon_input, num_points, distance_km):
    # Convert inputs to decimal degrees
    try:
        center_lat = dms_to_decimal(lat_input)
        center_lon = dms_to_decimal(lon_input)
    except ValueError as e:
        raise ValueError(f"Invalid coordinate format: {e}")

    vertices = []
    for i in range(num_points):
        bearing = (i * 360.0) / num_points
        lat, lon = get_point(center_lat, center_lon, distance_km, bearing)
        vertices.append((lat, lon))
    return vertices

In [8]:
def convert_vertices_to_geojson(vertices):
    """
    Convert polygon vertices to a GeoJSON-like Polygon dictionary.

    Args:
        vertices (list): List of (latitude, longitude) tuples

    Returns:
        dict: GeoJSON-like Polygon dictionary
    """
    # Convert (latitude, longitude) to (longitude, latitude) for GeoJSON
    coordinates = [[lon, lat] for lat, lon in vertices]

    # Ensure the polygon is closed by appending the first coordinate if necessary
    if coordinates:
        if coordinates[0] != coordinates[-1]:
            coordinates.append(coordinates[0])

    return {
        'type': 'Polygon',
        'coordinates': [coordinates]
    }

In [9]:
import folium
from folium import Figure
from html2image import Html2Image

def visualize_with_folium(vertices, center_lat=None, center_lon=None,
                          zoom_start=12, save_path="map.html"):
    """
    Creates a Folium map with a polygon and saves it as an HTML file.

    :param vertices: List of [lat, lon] pairs defining the polygon
    :param center_lat: Center latitude (optional)
    :param center_lon: Center longitude (optional)
    :param zoom_start: Initial zoom level
    :param save_path: Path to save the HTML file
    :return: Folium Map object
    """
    # Calculate center if not provided
    if center_lat is None or center_lon is None:
        center_lat = sum(v[0] for v in vertices) / len(vertices)
        center_lon = sum(v[1] for v in vertices) / len(vertices)

    # Create a Figure container
    figure = Figure(width=800, height=600)

    # Create base map inside the figure
    map_obj = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=zoom_start,
        tiles='Esri.WorldImagery'
    )
    figure.add_child(map_obj)

    # Add elements
    folium.Marker(
        [center_lat, center_lon],
        popup="Center",
        icon=folium.Icon(color='red')
    ).add_to(map_obj)

    folium.Polygon(
        vertices,
        color='#ff0000',
        fill=True,
        fill_opacity=0.2
    ).add_to(map_obj)

    # Save HTML
    map_obj.save(save_path)

    return map_obj

Copy coordinates from google earth

In [10]:
from mmap import MAP_ANON
# Generate vertices
vertices = generate_polygon_vertices(
    lat_input= LATITUDE,
    lon_input= LONGITUDE,
    num_points= POLIGON_VERTICES,
    distance_km= RADIO
)

# Define paths
map_name = f'{LOCATION_NAME}_map_{DESCRIPCION}'

# Define the output directory for CSV files
output_directory = os.path.join(os.getcwd(),map_name)

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

png_file_path = os.path.join(output_directory, f"{map_name}.png")
html_map_path = os.path.join(output_directory,f"{map_name}.html")


# Create visualization and save HTML
map_obj = visualize_with_folium(
    vertices=vertices,
    save_path=html_map_path,
    zoom_start=12
)



In [11]:
import asyncio
from playwright.async_api import async_playwright
import os
import nest_asyncio # Import nest_asyncio

# Apply nest_asyncio to allow asyncio to run inside a loop (like Jupyter)
nest_asyncio.apply()

async def take_screenshot(html_path, output_png_path, width=800, height=600):
    async with async_playwright() as p:
        browser = await p.chromium.launch() # Or firefox, webkit

        # --- Add this line to create a BrowserContext ---
        context = await browser.new_context(viewport={'width': width, 'height': height})
        # --- End of added line ---

        # --- Modify this line to create the page from the context ---
        page = await context.new_page() # Create the page within the context
        # --- End of modified line ---

        # Construct file URL
        file_url = f'file://{os.path.abspath(html_path)}'
        print(f"Attempting to load HTML file from: {file_url}") # Debugging print

        try:
            await page.goto(file_url)
            print(f"Successfully loaded {html_path}")
            await page.screenshot(path=output_png_path)
            print(f"Screenshot saved to: {output_png_path}")
        except Exception as e:
            print(f"Error during screenshot: {e}")
        finally:
            # --- Close the context and then the browser ---
            await context.close() # Close the context
            await browser.close() # Close the browser
            # --- End of closing lines ---
            print("Browser and context closed.")


In [12]:
# Now run the async function
# You can use asyncio.run() after applying nest_asyncio, or the %autoawait magic

# Using asyncio.run()
try:
    asyncio.run(take_screenshot(html_map_path, png_file_path, width=800, height=600))
except Exception as e:
     print(f"Error running asyncio task: {e}")


Attempting to load HTML file from: file:///content/Valle 3&4 (Valladolid, Spain)_map_Albedo 5km radio/Valle 3&4 (Valladolid, Spain)_map_Albedo 5km radio.html
Successfully loaded /content/Valle 3&4 (Valladolid, Spain)_map_Albedo 5km radio/Valle 3&4 (Valladolid, Spain)_map_Albedo 5km radio.html
Screenshot saved to: /content/Valle 3&4 (Valladolid, Spain)_map_Albedo 5km radio/Valle 3&4 (Valladolid, Spain)_map_Albedo 5km radio.png
Browser and context closed.


In [15]:
import datetime

# Define your Region of Interest (AOI) as a GeoJSON-like dictionary.
# Example: A small rectangle in an arbitrary location.
# REPLACE with your actual coordinates.
poligon_AOI = convert_vertices_to_geojson(vertices)
region_of_interest = poligon_AOI

# Define your date range for time-series data
example_start_date = INITIAL_DATE
example_end_date = FINAL_DATE

# Define the year for land cover analysis
example_year_lc = '2020'

# Define the desired frequency for time-series aggregation
# Options: 'hourly', 'daily', 'monthly', 'yearly'
example_frequency = FRECUENCY

# --- Ensure output directory exists ---
os.makedirs(output_directory, exist_ok=True) # Generic function handles this

# Extracting Albedo BSA
print(f"--- Extracting Albedo BSA ({example_frequency}) ---")
bsa_path = extract_albedo_bsa(
    region_geojson=region_of_interest,
    start_date_str=example_start_date,
    end_date_str=example_end_date,
    frequency=example_frequency,
    output_dir=output_directory
)

# Extracting Albedo WSA
print(f"--- Extracting Albedo WSA ({example_frequency}) ---")
wsa_path = extract_albedo_wsa(
    region_geojson=region_of_interest,
    start_date_str=example_start_date,
    end_date_str=example_end_date,
    frequency=example_frequency,
    output_dir=output_directory
)

# print(f"--- Extracting Approximated Solar Radiation ({example_frequency}) ---")
# extract_radiacion_solar(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency,
#     output_dir=output_directory
# )

# print(f"--- Extracting Day Temperature ({example_frequency}) ---")
# extract_temperatura_dia(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency,
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Night Temperature ({example_frequency}) ---")
# extract_temperatura_noche(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency,
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Wind Data ({example_frequency}) ---")
# # Note: Wind data from ERA5 is hourly. Generic function will average u/v components
# # to the target frequency before calculating speed/direction.
# extract_viento(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency, # e.g., 'daily' will average hourly components to daily means first
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Topography Data (Static) ---")
# extract_topography(
#     region_geojson=region_of_interest,
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Land Cover Data for {example_year_lc} ---")
# extract_land_cover(
#     region_geojson=region_of_interest,
#     year_str=example_year_lc,
#     output_dir=output_csv_directory
# )



Processing period 2001-01-01
Processing period 2001-01-02
Processing period 2001-01-03
Processing period 2001-01-04
Processing period 2001-01-05
Processing period 2001-01-06
Processing period 2001-01-07
Processing period 2001-01-08
Processing period 2001-01-09
Processing period 2001-01-10
Processing period 2001-01-11
Processing period 2001-01-12
Processing period 2001-01-13
Processing period 2001-01-14
Processing period 2001-01-15
Processing period 2001-01-16
Processing period 2001-01-17
Processing period 2001-01-18
Processing period 2001-01-19
Processing period 2001-01-20
Processing period 2001-01-21
Processing period 2001-01-22
Processing period 2001-01-23
Processing period 2001-01-24
Processing period 2001-01-25
Processing period 2001-01-26
Processing period 2001-01-27
Processing period 2001-01-28
Processing period 2001-01-29
Processing period 2001-01-30
Processing period 2001-01-31


Processing period 2001-02-01
Processing period 2001-02-02
Processing period 2001-02-03
Processing period 2001-02-04
Processing period 2001-02-05
Processing period 2001-02-06
Processing period 2001-02-07
Processing period 2001-02-08
Processing period 2001-02-09
Processing period 2001-02-10
Processing period 2001-02-11
Processing period 2001-02-12
Processing period 2001-02-13
Processing period 2001-02-14
Processing period 2001-02-15
Processing period 2001-02-16
Processing period 2001-02-17
Processing period 2001-02-18
Processing period 2001-02-19
Processing period 2001-02-20
Processing period 2001-02-21
Processing period 2001-02-22
Processing period 2001-02-23
Processing period 2001-02-24
Processing period 2001-02-25
Processing period 2001-02-26
Processing period 2001-02-27
Processing period 2001-02-28
Processing period 2001-03-01
Processing period 2001-03-02
Processing period 2001-03-03
Processing period 2001-03-04
Processing period 2001-03-05
Processing period 2001-03-06
Processing per

Processing period 2001-03-27
Processing period 2001-03-28
Processing period 2001-03-29
Processing period 2001-03-30
Processing period 2001-03-31
Processing period 2001-04-01
Processing period 2001-04-02
Processing period 2001-04-03
Processing period 2001-04-04
Processing period 2001-04-05
Processing period 2001-04-06
Processing period 2001-04-07
Processing period 2001-04-08
Processing period 2001-04-09
Processing period 2001-04-10
Processing period 2001-04-11
Processing period 2001-04-12
Processing period 2001-04-13
Processing period 2001-04-14
Processing period 2001-04-15
Processing period 2001-04-16
Processing period 2001-04-17
Processing period 2001-04-18
Processing period 2001-04-19
Processing period 2001-04-20
Processing period 2001-04-21
Processing period 2001-04-22
Processing period 2001-04-23
Processing period 2001-04-24
Processing period 2001-04-25
Processing period 2001-04-26
Processing period 2001-04-27
Processing period 2001-04-28
Processing period 2001-04-29
Processing per

Processing period 2001-11-13
Processing period 2001-11-14
Processing period 2001-11-15
Processing period 2001-11-16
Processing period 2001-11-17
Processing period 2001-11-18
Processing period 2001-11-19
Processing period 2001-11-20
Processing period 2001-11-21
Processing period 2001-11-22
Processing period 2001-11-23
Processing period 2001-11-24
Processing period 2001-11-25
Processing period 2001-11-26
Processing period 2001-11-27
Processing period 2001-11-28
Processing period 2001-11-29
Processing period 2001-11-30
Processing period 2001-12-01
Processing period 2001-12-02
Processing period 2001-12-03
Processing period 2001-12-04
Processing period 2001-12-05
Processing period 2001-12-06
Processing period 2001-12-07
Processing period 2001-12-08
Processing period 2001-12-09
Processing period 2001-12-10
Processing period 2001-12-11
Processing period 2001-12-12
Processing period 2001-12-13
Processing period 2001-12-14
Processing period 2001-12-15
Processing period 2001-12-16
Processing per

Processing period 2003-06-15
Processing period 2003-06-16
Processing period 2003-06-17
Processing period 2003-06-18
Processing period 2003-06-19
Processing period 2003-06-20
Processing period 2003-06-21
Processing period 2003-06-22
Processing period 2003-06-23
Processing period 2003-06-24
Processing period 2003-06-25
Processing period 2003-06-26
Processing period 2003-06-27
Processing period 2003-06-28
Processing period 2003-06-29
Processing period 2003-06-30
Processing period 2003-07-01
Processing period 2003-07-02
Processing period 2003-07-03
Processing period 2003-07-04
Processing period 2003-07-05
Processing period 2003-07-06
Processing period 2003-07-07
Processing period 2003-07-08
Processing period 2003-07-09
Processing period 2003-07-10
Processing period 2003-07-11
Processing period 2003-07-12
Processing period 2003-07-13
Processing period 2003-07-14
Processing period 2003-07-15
Processing period 2003-07-16
Processing period 2003-07-17
Processing period 2003-07-18
Processing per

Processing period 2003-09-21
Processing period 2003-09-22
Processing period 2003-09-23
Processing period 2003-09-24
Processing period 2003-09-25
Processing period 2003-09-26
Processing period 2003-09-27
Processing period 2003-09-28
Processing period 2003-09-29
Processing period 2003-09-30
Processing period 2003-10-01
Processing period 2003-10-02
Processing period 2003-10-03
Processing period 2003-10-04
Processing period 2003-10-05
Processing period 2003-10-06
Processing period 2003-10-07
Processing period 2003-10-08
Processing period 2003-10-09
Processing period 2003-10-10
Processing period 2003-10-11
Processing period 2003-10-12
Processing period 2003-10-13
Processing period 2003-10-14
Processing period 2003-10-15
Processing period 2003-10-16
Processing period 2003-10-17
Processing period 2003-10-18
Processing period 2003-10-19
Processing period 2003-10-20
Processing period 2003-10-21
Processing period 2003-10-22
Processing period 2003-10-23
Processing period 2003-10-24
Processing per

Processing period 2010-02-28
Processing period 2010-03-01
Processing period 2010-03-02
Processing period 2010-03-03
Processing period 2010-03-04
Processing period 2010-03-05
Processing period 2010-03-06
Processing period 2010-03-07
Processing period 2010-03-08
Processing period 2010-03-09
Processing period 2010-03-10
Processing period 2010-03-11
Processing period 2010-03-12
Processing period 2010-03-13
Processing period 2010-03-14
Processing period 2010-03-15
Processing period 2010-03-16
Processing period 2010-03-17
Processing period 2010-03-18
Processing period 2010-03-19
Processing period 2010-03-20
Processing period 2010-03-21
Processing period 2010-03-22
Processing period 2010-03-23
Processing period 2010-03-24
Processing period 2010-03-25
Processing period 2010-03-26
Processing period 2010-03-27
Processing period 2010-03-28
Processing period 2010-03-29
Processing period 2010-03-30
Processing period 2010-03-31
Processing period 2010-04-01
Processing period 2010-04-02
Processing per

Processing period 2010-04-26
Processing period 2010-04-27
Processing period 2010-04-28
Processing period 2010-04-29
Processing period 2010-04-30
Processing period 2010-05-01
Processing period 2010-05-02
Processing period 2010-05-03
Processing period 2010-05-04
Processing period 2010-05-05
Processing period 2010-05-06
Processing period 2010-05-07
Processing period 2010-05-08
Processing period 2010-05-09
Processing period 2010-05-10
Processing period 2010-05-11
Processing period 2010-05-12
Processing period 2010-05-13
Processing period 2010-05-14
Processing period 2010-05-15
Processing period 2010-05-16
Processing period 2010-05-17
Processing period 2010-05-18
Processing period 2010-05-19
Processing period 2010-05-20
Processing period 2010-05-21
Processing period 2010-05-22
Processing period 2010-05-23
Processing period 2010-05-24
Processing period 2010-05-25
Processing period 2010-05-26
Processing period 2010-05-27
Processing period 2010-05-28
Processing period 2010-05-29
Processing per

'/content/Valle 3&4 (Valladolid, Spain)_map_Albedo 5km radio/Radiacion_Solar_Approximated_from_LST_daily_20010101_20231130.csv'

In [ ]:
#Radiation functions

'''
Solar Radiation Extraction Functions for Earth Engine Datasets
------------------------------------------------------------

Provides functions to extract solar radiation data from three primary sources:

1. **ECMWF ERA5-Land Hourly (Surface Solar Radiation)**
2. **NASA GLDAS (Shortwave Radiation)**
3. **MODIS MCD18A1 (Direct/Global Solar Radiation)**

================================================================================
Key Features & Usage Notes
================================================================================

### 1. ECMWF ERA5-Land (Hourly Surface Solar Radiation)
- **Band**: `surface_solar_radiation_downwards` (cumulative J/m²)
- **Post-processing**: Converts to average W/m² for the hour
- **Scale**: Native 10km resolution (`scale=10000`)
- **Temporal Resolution**: Hourly
- **Data Availability**: 1940-present (near real-time)
- **Cloud Handling**: Model-based (no gaps)
- **Output Unit**: W/m²
- **Best For**: Historical analysis, hourly energy modeling

### 2. NASA GLDAS (3-Hourly Shortwave Radiation)
- **Band**: `SWnet_avg` (net shortwave radiation in W/m²)
- **Post-processing**: None (directly usable values)
- **Scale**: Native 25km resolution (`scale=25000`)
- **Temporal Resolution**: 3-hourly
- **Data Availability**: 2000-present
- **Cloud Handling**: Model-based (no gaps)
- **Output Unit**: W/m²
- **Best For**: Energy balance studies, land surface modeling

### 3. MODIS MCD18A1 (Daily Solar Radiation)
- **Band Options**:
  - `SWGDN`: Global downwelling shortwave (W/m²)
  - `SWTDN`: Direct downwelling shortwave (W/m²)
  - `SWGDNI`: Diffuse downwelling shortwave (W/m²)
- **Post-processing**:
  - Applies scale factor (0.1)
  - Filters fill values (≥32767 = NaN)
- **Scale**: Native 1km resolution (`scale=1000`)
- **Temporal Resolution**: Daily
- **Data Availability**: 2000-2021 (verify current status)
- **Cloud Handling**: Contains gaps due to clouds
- **Output Unit**: W/m²
- **Best For**: High-resolution daily analysis, regional studies

================================================================================
Important Considerations
================================================================================

### Temporal Resolution Compatibility
- Match frequency argument to dataset capabilities:
  - ERA5: Supports `"hourly"`, `"daily"`, `"monthly"`
  - GLDAS: Supports `"3-hourly"`, `"daily"`, `"monthly"`
  - MODIS: Supports `"daily"`, `"monthly"` (not hourly)

### Data Availability
- **ERA5**: 1940 to near real-time (~3-5 day latency)
- **GLDAS**: 2000 to present (~1-2 month latency)
- **MODIS**: 2000-2021 (check for updates)

### Data Quality Notes
- **MODIS**:
  - Requires fill value filtering (≥32767 = NaN)
  - Contains gaps from cloud cover
  - Daily composites may have missing pixels
- **Model Data (ERA5/GLDAS)**:
  - Complete spatiotemporal coverage
  - Lower spatial resolution than MODIS
  - Potential bias in complex terrain

### Unit Consistency
- All final outputs converted to **W/m²**
- MODIS requires scaling (0.1 multiplier)
- ERA5 requires conversion (J/m² → W/m²)

### Processing Recommendations
- **Large Regions**: Start with coarse resolution (ERA5/GLDAS)
- **High Resolution**: Use MODIS but expect missing data
- **Time-Sensitive Projects**: Prefer ERA5 for near real-time
- **Historical Analysis**: All datasets suitable within date ranges
'''


import numpy as np

# 1. ECMWF ERA5-Land Hourly - Surface Solar Radiation
def extract_era5_solar_radiation(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=10000):
    """
    Extracts surface solar radiation (SSR) from ERA5-Land dataset.
    SSR: Cumulative energy in J/m² (convert to W/m² by dividing by 3600 for hourly)
    """
    return extract_gee_time_series(
        variable_name="Surface_Solar_Radiation",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='ECMWF/ERA5_LAND/HOURLY',
        gee_band_name='surface_solar_radiation_downwards',
        scale=scale,
        output_dir=output_dir,
        post_process_function=post_process_era5_radiation,
        nan_value=np.nan
    )


def extract_era5_ghi_dhi(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=10000):
    """
    Extrae radiación solar global (GHI) y difusa (DHI) del dataset ERA5 de ECMWF

    Parámetros:
    region_geojson: GeoJSON del área de interés
    start_date_str: Fecha inicio (YYYY-MM-DD)
    end_date_str: Fecha fin (YYYY-MM-DD)
    frequency: Frecuencia de agregación ('hourly', 'daily', 'monthly')
    output_dir: Directorio de salida
    scale: Resolución en metros (nativa=10000)
    """
    return extract_gee_time_series(
        variable_names=["GHI", "DHI"],
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='ECMWF/ERA5/HOURLY',
        gee_band_names=[
            'surface_solar_radiation_downwards',  # GHI
            'surface_diffuse_downwelling_shortwave_radiation_in_air'  # DHI
        ],
        scale=scale,
        output_dir=output_dir,
        post_process_function=post_process_era5_radiation,
        nan_value=np.nan
    )

def era5_radiation_conversion(raw_values):
    """Converts ERA5 radiation from J/m² to W/m²"""
    return {
        'GHI': raw_values['surface_solar_radiation_downwards'] / 3600,
        'DHI': raw_values['surface_diffuse_downwelling_shortwave_radiation_in_air'] / 3600
    }



def post_process_era5_radiation(values_dict):
    """Convierte valores acumulados (J/m²) a W/m²"""
    processed = {}
    for band, value in values_dict.items():
        if value is None:
            processed[band] = np.nan
        else:
            # Conversión J/m² → W/m²
            processed[band] = value / 3600
    return processed

def post_process_era5_radiation(ssr_value):
    """Convert ERA5 cumulative radiation (J/m²) to average W/m²"""
    if ssr_value is None:
        return None
    # Hourly data: J/m² → W/m² conversion (1 W = 1 J/s → divide by 3600 seconds)
    return ssr_value / 3600 if ssr_value >= 0 else np.nan


# 2. NASA GLDAS - Shortwave Radiation
def extract_gldas_shortwave_radiation(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=25000):
    """
    Extracts surface incoming shortwave radiation (SW_net) from GLDAS dataset.
    Directly in W/m² (no conversion needed)
    """
    return extract_gee_time_series(
        variable_name="Shortwave_Radiation",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='NASA/GLDAS/V021/NOAH/G025/T3H',
        gee_band_name='SWnet_avg',  # Net shortwave radiation (W/m²)
        scale=scale,
        output_dir=output_dir,
        post_process_function=None,  # No conversion needed
        nan_value=np.nan
    )


# 3. MODIS MCD18A1 - Direct/Global Solar Radiation
def extract_modis_solar_radiation(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=1000):
    """
    Extracts MODIS solar radiation products (daily averages)
    Options: 'SWGDN' (Global), 'SWTDN' (Direct), or 'SWGDNI' (Diffuse)
    """
    return extract_gee_time_series(
        variable_name="MODIS_Solar_Radiation",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/006/MCD18A1',
        gee_band_name='SWGDN',  # Global Downwelling Shortwave (W/m²)
        scale=scale,
        output_dir=output_dir,
        post_process_function=post_process_modis_radiation,
        nan_value=np.nan
    )

def post_process_modis_radiation(swgdn_value):
    """Handle MODIS fill values and scaling"""
    if swgdn_value is None:
        return None
    # MODIS uses 32767 as fill value for missing data
    if swgdn_value >= 32700:  # All fill values are >= 32767
        return np.nan
    # Scale factor is 0.1 (W/m² per unit)
    return swgdn_value * 0.1





In [ ]:
# 1. ECMWF ERA5-Land Hourly - Surface Solar Radiation
extract_era5_solar_radiation(
    region_of_interest,
    "2020-01-01",
    "2020-01-31",
    "hourly",
    output_directory
)


# 2. NASA GLDAS - Shortwave Radiation
extract_gldas_shortwave_radiation(
    region_geojson=region_of_interest,
    start_date_str="2023-01-01",
    end_date_str="2023-12-31",
    frequency="monthly",
    output_dir=output_directory,
    scale=25000  # Native 25km resolution
)

#2. Example for Annual summary for multiple years:
for year in range(2020, 2024):
    extract_gldas_shortwave_radiation(
        region_geojson=region_of_interest,
        start_date_str=f"{year}-01-01",
        end_date_str=f"{year}-12-31",
        frequency="annual",
        output_dir=f"./annual_results/{year}"
    )

# 3. MODIS MCD18A1 - Direct/Global Solar Radiation
extract_modis_solar_radiation(
    region_of_interest,
    "2020-01-01",
    "2020-12-31",
    "monthly",
    output_directory,
    scale=1000
)

In [ ]:
def era5_radiation_conversion(raw_values):
    """Converts ERA5 radiation from J/m² to W/m²"""
    return {
        'GHI': raw_values['surface_solar_radiation_downwards'] / 3600,
        'DHI': raw_values['surface_diffuse_solar_radiation_downwards'] / 3600  # Nombre corregido
    }

# Extraer datos horarios usando ERA5 completo (no Land)
csv_path = extract_gee_time_series_multi(
    variable_names=['GHI', 'DHI'],
    region_geojson=region_of_interest,
    start_date_str="2023-01-01",
    end_date_str="2023-01-31",
    frequency="hourly",
    gee_project=PROJECT,
    gee_dataset_id='ECMWF/ERA5/HOURLY',  # Dataset corregido
    gee_band_names=[
        'surface_solar_radiation_downwards',       # GHI
        'surface_diffuse_solar_radiation_downwards'  # DHI (nombre correcto)
    ],
    scale=31000,  # Resolución nativa de ERA5 (31km)
    output_dir="./radiation_data",
    post_process_function=era5_radiation_conversion,
    nan_value=np.nan
)

Processing period 2023-01-01 00:00:00
Error processing period 2023-01-01 00:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-01 01:00:00
Error processing period 2023-01-01 01:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-01 02:00:00
Error processing period 2023-01-01 02:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-01 03:00:00
Error processing period 2023-01-01 03:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-01 04:00:00
Error processing period 2023-01-01 04:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-01 05:00:00
Error processing period 2023-01-01 05:00:00: Dictionary.get: Dictionary does 

Error processing period 2023-01-09 19:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-09 20:00:00
Error processing period 2023-01-09 20:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-09 21:00:00
Error processing period 2023-01-09 21:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-09 22:00:00
Error processing period 2023-01-09 22:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-09 23:00:00
Error processing period 2023-01-09 23:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_solar_radiation_downwards'.
Processing period 2023-01-10 00:00:00
Error processing period 2023-01-10 00:00:00: Dictionary.get: Dictionary does not contain key: 'surface_diffuse_sola

In [ ]:
# 3. MODIS MCD18A1 - Direct/Global Solar Radiation
extract_modis_solar_radiation(
    region_of_interest,
    "2020-01-01",
    "2020-12-31",
    "monthly",
    output_directory,
    scale=1000
)

Processing period 2020-01
Error processing period 2020-01 for MODIS_Solar_Radiation: ImageCollection.load: ImageCollection asset 'MODIS/006/MCD18A1' not found (does not exist or caller does not have access).
Processing period 2020-02
Error processing period 2020-02 for MODIS_Solar_Radiation: ImageCollection.load: ImageCollection asset 'MODIS/006/MCD18A1' not found (does not exist or caller does not have access).
Processing period 2020-03
Error processing period 2020-03 for MODIS_Solar_Radiation: ImageCollection.load: ImageCollection asset 'MODIS/006/MCD18A1' not found (does not exist or caller does not have access).
Processing period 2020-04
Error processing period 2020-04 for MODIS_Solar_Radiation: ImageCollection.load: ImageCollection asset 'MODIS/006/MCD18A1' not found (does not exist or caller does not have access).
Processing period 2020-05
Error processing period 2020-05 for MODIS_Solar_Radiation: ImageCollection.load: ImageCollection asset 'MODIS/006/MCD18A1' not found (does not

'/content/Phofu (Vierfontein, South Africa)_map_Radiation 5km radio/MODIS_Solar_Radiation_monthly_20200101_20201231.csv'

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.units import inch
from datetime import datetime

# 1. Load and preprocess data
def load_data(bsa_path, wsa_path):
    bsa = pd.read_csv(bsa_path, parse_dates=['timestamp'])
    wsa = pd.read_csv(wsa_path, parse_dates=['timestamp'])
    # Merge BSA and WSA data
    df = pd.merge(bsa, wsa, on='timestamp', suffixes=('_BSA', '_WSA'))
    df['month'] = df['timestamp'].dt.month
    df['year'] = df['timestamp'].dt.year
    return df

# 2. Generate statistics tables
def calculate_statistics(df):
    # Calculate monthly statistics for Albedo BSA
    monthly_stats_bsa = df.groupby('month')['Albedo_BSA'].agg([
        'mean', 'min', 'max',
        lambda x: x.quantile(0.01),  # p1
        lambda x: x.quantile(0.05),  # p5
        lambda x: x.quantile(0.10),  # p10
        lambda x: x.quantile(0.25),  # p25
        lambda x: x.quantile(0.50),  # median
        lambda x: x.quantile(0.75),  # p75
        lambda x: x.quantile(0.90),  # p90
        lambda x: x.quantile(0.95),  # p95
        lambda x: x.quantile(0.99),  # p99
    ])
    monthly_stats_bsa.columns = ['average', 'min', 'max', 'p1', 'p5', 'p10', 'p25',
                                'p50', 'p75', 'p90', 'p95', 'p99']

    # Calculate monthly statistics for Albedo WSA (New)
    monthly_stats_wsa = df.groupby('month')['Albedo_WSA'].agg([
        'mean', 'min', 'max',
        lambda x: x.quantile(0.01),  # p1
        lambda x: x.quantile(0.05),  # p5
        lambda x: x.quantile(0.10),  # p10
        lambda x: x.quantile(0.25),  # p25
        lambda x: x.quantile(0.50),  # median
        lambda x: x.quantile(0.75),  # p75
        lambda x: x.quantile(0.90),  # p90
        lambda x: x.quantile(0.95),  # p95
        lambda x: x.quantile(0.99),  # p99
    ])
    monthly_stats_wsa.columns = ['average', 'min', 'max', 'p1', 'p5', 'p10', 'p25',
                                'p50', 'p75', 'p90', 'p95', 'p99']

    # Return both sets of statistics
    return monthly_stats_bsa, monthly_stats_wsa

# 3. Create visualizations
def create_figures(df, monthly_stats_bsa, monthly_stats_wsa):
    # Figure 1: Monthly statistics (BSA)
    plt.figure(figsize=(10, 6))
    monthly_stats_bsa[['average', 'min', 'max']].plot(kind='line', marker='o')
    plt.title('Monthly Albedo BSA Statistics') # Updated title
    plt.ylabel('Albedo BSA') # Updated label
    plt.xlabel('Month')
    plt.grid(True)
    plt.savefig('monthly_stats_bsa.png') # Updated filename
    plt.close()

    # Figure 2: Daily values timeline (BSA)
    plt.figure(figsize=(12, 6))
    plt.plot(df['timestamp'], df['Albedo_BSA'], label='Daily Albedo BSA') # Updated label
    plt.title('Daily Albedo BSA Values') # Updated title
    plt.ylabel('Albedo BSA') # Updated label
    plt.xlabel('Date')
    plt.grid(True)
    plt.savefig('daily_values_bsa.png') # Updated filename
    plt.close()

    # Figure 3: Monthly statistics (WSA) (New)
    plt.figure(figsize=(10, 6))
    monthly_stats_wsa[['average', 'min', 'max']].plot(kind='line', marker='o')
    plt.title('Monthly Albedo WSA Statistics') # New title
    plt.ylabel('Albedo WSA') # New label
    plt.xlabel('Month')
    plt.grid(True)
    plt.savefig('monthly_stats_wsa.png') # New filename
    plt.close()

    # Figure 4: Daily values timeline (WSA) (New)
    plt.figure(figsize=(12, 6))
    plt.plot(df['timestamp'], df['Albedo_WSA'], label='Daily Albedo WSA') # New label
    plt.title('Daily Albedo WSA Values') # New title
    plt.ylabel('Albedo WSA') # New label
    plt.xlabel('Date')
    plt.grid(True)
    plt.savefig('daily_values_wsa.png') # New filename
    plt.close()


# 4. Generate PDF report
def create_pdf_report(monthly_stats_bsa, monthly_stats_wsa, annual_avg_bsa, annual_avg_wsa, df):

    # Define the full path for the PDF using the output_directory variable
    pdf_filename = f"{LOCATION_NAME}_Albedo_Report_{DESCRIPCION}.pdf"
    pdf_path = os.path.join(output_directory, pdf_filename) # Use os.path.join for path safety

    # Instantiate SimpleDocTemplate with the full path
    doc = SimpleDocTemplate(pdf_path, pagesize=letter)
    styles = getSampleStyleSheet()
    elements = []

    # Add logo if URL is provided
    logo_url = 'https://coxenergy.es/img/logos/logo-header.png'
    logo_cell = ''
    if logo_url:
        try:
            # ReportLab's Image object can take a URL directly
            logo_img = Image(logo_url, width=(64/100)*inch, height=(39/100)*inch, hAlign='LEFT') # Adjust width/height as needed
            logo_cell = logo_img
            print(f"Loading logo from URL: {logo_url}")
            elements.append(logo_cell)
        except Exception as e:
            print(f"Could not load logo from URL {logo_url}: {e}")
            # Optionally add a placeholder or skip the logo


    # Title
    elements.append(Paragraph(f"{LOCATION_NAME}", styles['Title']))
    elements.append(Paragraph(f"Albedo Analysis Report", styles['Title']))
    elements.append(Paragraph(f"{DESCRIPCION} ", styles['Title']))
    elements.append(Spacer(1, 0.1*inch))

    # Add map image
    # Ensure poligon_AOI is accessible or passed as an argument if needed
    try:
        coordinates_text = f'{LATITUDE} - {LONGITUDE} \nPoligon {POLIGON_VERTICES} vertices, \nradio {RADIO}[Km]'
    except NameError:
        coordinates_text = "Coordinates not available" # Handle case if poligon_AOI is not defined globally

    elements.append(Paragraph("Study Area Location", styles['Heading2']))
    elements.append(Paragraph(coordinates_text, styles['BodyText']))
    elements.append(Spacer(1, 0.1*inch))

    # Check if the image file exists before adding it
    map_image = os.path.join(output_directory, f"{map_name}.png")
    if map_image:
        elements.append(Image(map_image, width=6*inch, height=4*inch))
    else:
        elements.append(Paragraph("Map image (map.png) not found.", styles['BodyText']))

    elements.append(Spacer(1, 0.1*inch))


    # Methodology
    elements.append(Paragraph("Methodology", styles['Heading2'])) # Add a heading for the methodology

    # Create separate Paragraph objects for each block of text
    methodology_paragraph_1 = """
    <para align=justify>
    The albedo analysis uses MODIS MCD43A3 data (Collection 6.1), including both
    black-sky albedo (BSA) and white-sky albedo (WSA) values, aggregated to a daily
    frequency. The analysis covers the period from {start_date} to {end_date}.
    </para>
    """.format(
        start_date=df['timestamp'].min().strftime('%Y-%m-%d'), # Use df to get actual date range
        end_date=df['timestamp'].max().strftime('%Y-%m-%d')   # Use df to get actual date range
    )
    elements.append(Paragraph(methodology_paragraph_1, styles['BodyText']))
    elements.append(Spacer(1, 0.25*inch)) # Add a small space between paragraphs

    methodology_paragraph_2 = """
    <para align=justify>
    For BSA, values range between {min_val_bsa:.2f} and {max_val_bsa:.2f} with an
    overall average of {avg_val_bsa:.2f}. For WSA, values range between {min_val_wsa:.2f}
    and {max_val_wsa:.2f} with an overall average of {avg_val_wsa:.2f}.
    </para>
    """.format(
        min_val_bsa=df['Albedo_BSA'].min(),
        max_val_bsa=df['Albedo_BSA'].max(),
        avg_val_bsa=annual_avg_bsa,
        min_val_wsa=df['Albedo_WSA'].min(),
        max_val_wsa=df['Albedo_WSA'].max(),
        avg_val_wsa=annual_avg_wsa
    )
    elements.append(Paragraph(methodology_paragraph_2, styles['BodyText']))
    elements.append(PageBreak()) # This line forces a new page after the spacer

    # Monthly Statistics Table (BSA)
    elements.append(Paragraph("Monthly Albedo BSA Statistics", styles['Heading2'])) # New heading
    table_data_bsa = [['Month', 'Avg', 'Min', 'Max', 'P10', 'P50', 'P90']]
    for month, data in monthly_stats_bsa.iterrows():
        table_data_bsa.append([
            str(month),
            f"{data['average']:.2f}",
            f"{data['min']:.2f}",
            f"{data['max']:.2f}",
            f"{data['p10']:.2f}",
            f"{data['p50']:.2f}",
            f"{data['p90']:.2f}"
        ])

    t_bsa = Table(table_data_bsa)
    t_bsa.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.grey),
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 12),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('BACKGROUND', (0,1), (-1,-1), colors.beige),
        ('GRID', (0,0), (-1,-1), 1, colors.black)
    ]))
    elements.append(t_bsa)
    elements.append(Spacer(1, 0.1*inch))

    # Add figures (BSA)
    elements.append(Paragraph("Monthly Albedo BSA Statistics Plot", styles['Heading2'])) # Updated heading
    # Check if image file exists
    if os.path.exists('monthly_stats_bsa.png'):
        elements.append(Image('monthly_stats_bsa.png', width=6*inch, height=4*inch)) # Updated filename
    else:
        elements.append(Paragraph("Monthly BSA statistics plot (monthly_stats_bsa.png) not found.", styles['BodyText']))

    elements.append(Spacer(1, 0.1*inch))
    elements.append(PageBreak()) # This line forces a new page after the spacer


    elements.append(Paragraph("Daily Albedo BSA Values Timeline Plot", styles['Heading2'])) # Updated heading
    # Check if image file exists
    if os.path.exists('daily_values_bsa.png'):
        elements.append(Image('daily_values_bsa.png', width=6*inch, height=4*inch)) # Updated filename
    else:
        elements.append(Paragraph("Daily BSA values timeline plot (daily_values_bsa.png) not found.", styles['BodyText']))
    elements.append(Spacer(1, 0.1*inch)) # Added space before next section
    elements.append(PageBreak()) # This line forces a new page after the spacer




    # Monthly Statistics Table (WSA) (New)
    elements.append(Paragraph("Monthly Albedo WSA Statistics", styles['Heading2'])) # New heading
    table_data_wsa = [['Month', 'Avg', 'Min', 'Max', 'P10', 'P50', 'P90']]
    for month, data in monthly_stats_wsa.iterrows():
        table_data_wsa.append([
            str(month),
            f"{data['average']:.2f}",
            f"{data['min']:.2f}",
            f"{data['max']:.2f}",
            f"{data['p10']:.2f}",
            f"{data['p50']:.2f}",
            f"{data['p90']:.2f}"
        ])

    t_wsa = Table(table_data_wsa)
    t_wsa.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.grey),
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 12),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('BACKGROUND', (0,1), (-1,-1), colors.beige),
        ('GRID', (0,0), (-1,-1), 1, colors.black)
    ]))
    elements.append(t_wsa)
    elements.append(Spacer(1, 0.1*inch))
    elements.append(Spacer(1, 0.1*inch))

    # Add figures (WSA) (New)
    elements.append(Paragraph("Monthly Albedo WSA Statistics Plot", styles['Heading2'])) # New heading
    # Check if image file exists
    if os.path.exists('monthly_stats_wsa.png'):
        elements.append(Image('monthly_stats_wsa.png', width=6*inch, height=4*inch)) # New filename
    else:
        elements.append(Paragraph("Monthly WSA statistics plot (monthly_stats_wsa.png) not found.", styles['BodyText']))
    elements.append(Spacer(1, 0.1*inch))

    elements.append(Paragraph("Daily Albedo WSA Values Timeline Plot", styles['Heading2'])) # New heading
    # Check if image file exists
    if os.path.exists('daily_values_wsa.png'):
        elements.append(Image('daily_values_wsa.png', width=6*inch, height=4*inch)) # New filename
    else:
        elements.append(Paragraph("Daily WSA values timeline plot (daily_values_wsa.png) not found.", styles['BodyText']))
    elements.append(Spacer(1, 0.1*inch)) # Added space at the end


    doc.build(elements)

# Main workflow
if __name__ == "__main__":
    # Load your CSV files
    try:
      df = load_data(bsa_path, wsa_path)
    except Exception as e: # Catch broader exception to see error
      print(f"Initial load failed: {e}. Attempting fallback paths.")
      # Fallback paths - make sure these match the filenames generated by extract_gee_time_series
      fallback_bsa_path = '/content/gee_output_data/Albedo_BSA_daily_20000101_20231130.csv'
      fallback_wsa_path = '/content/gee_output_data/Albedo_WSA_daily_20000101_20231130.csv'
      try:
         df = load_data(fallback_bsa_path, fallback_wsa_path)
         print("Fallback load successful.")
      except Exception as e_fallback:
         print(f"Fallback load failed: {e_fallback}. Cannot proceed with report generation.")
         df = None # Ensure df is None if loading fails
         # You might want to exit or handle this failure case appropriately
         # For now, the rest of the code will likely raise errors if df is None/empty.


    if df is not None and not df.empty: # Only proceed if data was loaded successfully
      # Calculate statistics for both BSA and WSA
      monthly_stats_bsa, monthly_stats_wsa = calculate_statistics(df) # Now returns two dataframes

      # Calculate annual averages for both BSA and WSA
      annual_avg_bsa = df['Albedo_BSA'].mean()
      annual_avg_wsa = df['Albedo_WSA'].mean() # New

      # Generate visualizations for both BSA and WSA
      create_figures(df, monthly_stats_bsa, monthly_stats_wsa) # Pass both stats dataframes

      # Create PDF report, passing all necessary data
      create_pdf_report(monthly_stats_bsa, monthly_stats_wsa, annual_avg_bsa, annual_avg_wsa, df) # Pass all data
      print("Albedo Report generated successfully.")

    else:
      print("Skipping report generation due to data loading failure.")

Initial load failed: name 'bsa_path' is not defined. Attempting fallback paths.
Fallback load failed: [Errno 2] No such file or directory: '/content/gee_output_data/Albedo_BSA_daily_20000101_20231130.csv'. Cannot proceed with report generation.
Skipping report generation due to data loading failure.


In [ ]:
# prompt: descargar  de la session de colab en un zip los archivos {/content/map.html
# /content/map.png} y el contenido de la carpeta {/content/gee_output_data}

!zip -r El Sanate_Guatemala_0_5km_radio.zip "/content/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio"

	zip warning: name not matched: Sanate_Guatemala_0_5km_radio.zip
  adding: content/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio/ (stored 0%)
  adding: content/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio/El Sanate (Guatemala)_Albedo_Report_Albedo Ground 0,5km radio.pdf (deflated 19%)
  adding: content/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio/Albedo_BSA_daily_20010101_20231130.csv (deflated 80%)
  adding: content/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio.html (deflated 66%)
  adding: content/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio.png (deflated 0%)
  adding: content/El Sanate (Guatemala)_map_Albedo Ground 0,5km radio/Albedo_WSA_daily_20010101_20231130.csv (deflated 79%)


In [ ]:
import ee
import matplotlib.pyplot as plt

# Definir la región de interés (Dubái)
# DEWA
# 24.76959, 55.441132
# 24.770681, 55.516148
# 24.739036, 55.516491
# 24.73545, 55.438042

dubai = ee.Geometry.Rectangle([24.76959, 55.441132, 24.739036, 55.516491])

# Rango de fechas para 2024
start = ee.Date('2024-01-01')
end = ee.Date('2024-12-31')

# Cargar la colección de GOES-16
# Nota: Asegúrate de que el producto esté disponible en GEE
goes = ee.ImageCollection('NOAA/GOES/16/MCMIPF') \  # Verifica el ID exacto del producto
    .filterDate(start, end) \
    .filterBounds(dubai)

# Función para extraer la máscara de nubes
def add_cloud_mask(image):
    # Supongamos que el producto tiene una banda de máscara de nubes llamada 'CMI'
    cloud_mask = image.select('CMI').gt(0.5)  # Ajusta el umbral según el producto
    cloud_pct = cloud_mask.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=dubai,
        scale=1000,
        maxPixels=1e13
    ).get('CMI')

    return image.set('cloud_coverage', cloud_pct)

# Aplicar la función a cada imagen
cloud_stats = goes.map(add_cloud_mask)

# Función para extraer la hora de cada imagen y su cobertura de nubes
def extract_hour_feature(image):
    datetime = image.date()
    hour = datetime.get('hour')
    cloud_coverage = image.get('cloud_coverage')
    return ee.Feature(None, {'hour': hour, 'cloud_coverage': cloud_coverage})

# Mapear la función sobre la colección
hourly_features = cloud_stats.map(extract_hour_feature)

# Convertir a FeatureCollection
hourly_cloud_fc = ee.FeatureCollection(hourly_features)

# Agrupar por hora y calcular el promedio horario
def calculate_hourly_mean(hour):
    hourly = hourly_cloud_fc.filter(ee.Filter.eq('hour', hour))
    mean = hourly.reduceColumns(ee.Reducer.mean(), ['cloud_coverage']).get('mean')
    return ee.Feature(None, {'hour': hour, 'cloud_coverage': mean})

hours = list(range(0, 24))
hourly_mean_features = [calculate_hourly_mean(h) for h in hours]
hourly_mean_fc = ee.FeatureCollection(hourly_mean_features)

# Obtener los resultados como un diccionario
results = hourly_mean_fc.reduceColumns(ee.Reducer.toList(2), ['hour', 'cloud_coverage']).get('list').getInfo()

# Separar horas y coberturas de nubes
hours_list = [feature[0] for feature in results]
cloud_coverage_list = [feature[1] for feature in results]

# Imprimir resultados
print('Cobertura de nubes por hora en Dubái (2024):')
for h, c in zip(hours_list, cloud_coverage_list):
    print(f'Hora {h}: {c*100:.2f}%')

# Graficar
plt.figure(figsize=(10,6))
plt.plot(hours_list, [c*100 for c in cloud_coverage_list], marker='o', linewidth=2)
plt.title('Cobertura de Nubes Horaria en Dubái (2024)')
plt.xlabel('Hora')
plt.ylabel('Cobertura de Nubes (%)')
plt.grid(True)
plt.xticks(hours_list)
plt.show()

SyntaxError: unexpected character after line continuation character (<ipython-input-34-0927d2f0fba1>, line 19)

In [ ]:
import requests
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from io import BytesIO
import logging
import concurrent.futures

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_cloud_status(current_time, data_url, username, password, latitude, longitude):
    try:
        response = requests.get(data_url, auth=(username, password))
        response.raise_for_status()
        nc_data = xr.open_dataset(BytesIO(response.content))
        cloud_mask = nc_data['cloud_mask']  # Update based on actual variable name
        nearest = cloud_mask.sel(latitude=latitude, longitude=longitude, method='nearest')
        cloud_value = nearest.item()
        if cloud_value in [1, 2]:
            status = 'clear'
        elif cloud_value == 3:
            status = 'cloudy'
        else:
            status = 'not processed'
        return {'datetime': current_time, 'cloud_status': status}
    except requests.HTTPError as http_err:
        print(f"HTTP error for {current_time}: {http_err}")
        return {'datetime': current_time, 'cloud_status': 'HTTP error'}
    except KeyError as key_err:
        print(f"Missing variable in NetCDF file for {current_time}: {key_err}")
        return {'datetime': current_time, 'cloud_status': 'Data error'}
    except Exception as err:
        print(f"Unexpected error for {current_time}: {err}")
        return {'datetime': current_time, 'cloud_status': 'error'}

def get_cloud_cover(latitude, longitude, start_time, end_time, username, password, api_url_template):
    """
    Retrieve cloud cover from EUMETSAT Data Store - Cloud Mask - MSG - Indian Ocean
    for a given latitude, longitude, and time period.

    Parameters:
    - latitude (float): Latitude of the location (degrees, e.g., -30.0).
    - longitude (float): Longitude of the location (degrees, e.g., 70.0).
    - start_time (str or datetime): Start of the time period (format: 'YYYY-MM-DD').
    - end_time (str or datetime): End of the time period (format: 'YYYY-MM-DD').
    - username (str): EUMETSAT Data Store username.
    - password (str): EUMETSAT Data Store password.
    - api_url_template (str): API endpoint template with placeholders for date and time.

    Returns:
    - pandas.DataFrame: DataFrame with 'datetime' and 'cloud_status' columns.
    """
    # Convert start and end times to datetime objects if they are strings
    if isinstance(start_time, str):
        start_time = datetime.strptime(start_time, "%Y-%m-%d")
    if isinstance(end_time, str):
        end_time = datetime.strptime(end_time, "%Y-%m-%d")

    # Initialize list to store results
    results = []

    # Define the time step based on data frequency
    time_step = timedelta(hours=1)  # Adjust if necessary

    # Prepare list of all times
    times = []
    current_time = start_time
    while current_time <= end_time:
        times.append(current_time)
        current_time += time_step

    # Function for concurrent execution
    def process_time(ct):
        date_str = ct.strftime("%Y%m%d")
        time_str = ct.strftime("%H%M")
        data_url = api_url_template.format(date=date_str, time=time_str)
        return fetch_cloud_status(ct, data_url, username, password, latitude, longitude)

    # Use ThreadPoolExecutor for I/O-bound tasks
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(process_time, ct): ct for ct in times}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            results.append(result)

    # Create a pandas DataFrame from the results
    df = pd.DataFrame(results)
    df.sort_values('datetime', inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [ ]:
# Example usage
latitude = 55.439335857996824       # Replace with your desired latitude
longitude = 24.722963564208747       # Replace with your desired longitude
start_date = '2023-09-01'  # Start of the period
end_date = '2023-09-07'    # End of the period
username = 'chinso' # EUMETSAT username
password = 'jbg4jaa-GP6D-xV' # EUMETSAT password
api_url_template = "https://api.eumetsat.int/dataset/EO:EUM:DAT:MSG1:MSGCLMK/data/{date}/{time}.nc"  # Update with actual endpoint





cloud_cover_df = get_cloud_cover(latitude, longitude, start_date, end_date, username, password, api_url_template)
print(cloud_cover_df)

Retrieve cloud cover from EUMETSAT Data Store - Cloud Mask - MSG - Indian Ocean
    using WEkEO HDA API for a given latitude, longitude, and time period.

In [ ]:
!pip install hda
!pip install cfgrib

In [ ]:
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import os
import time
import glob
import sys
import zipfile

# Try to import WEkEO client - handle if not installed
try:
    from hda import Client, Configuration
    HDA_AVAILABLE = True
except ImportError:
    print("WEkEO HDA client not installed. Install with: pip install wekeo-hda")
    HDA_AVAILABLE = False

# Import cfgrib for GRIB file handling
try:
    import cfgrib
    CFGRIB_AVAILABLE = True
except ImportError:
    print("cfgrib not installed. Install with: pip install cfgrib")
    CFGRIB_AVAILABLE = False


def create_download_folder(base_folder="wekeo_downloads"):
    """Create and return path to download folder"""
    # Create folder based on current date and time for organization
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    download_folder = os.path.join(base_folder, f"download_{timestamp}")

    # Create the directory if it doesn't exist
    os.makedirs(download_folder, exist_ok=True)
    print(f"Created download folder: {download_folder}")

    return download_folder

def setup_wekeo_client(username, password):
    """Initialize and return WEkEO client with credentials"""
    if not HDA_AVAILABLE:
        raise ImportError("WEkEO HDA client not available. Install with: pip install wekeo-hda")

    try:
        config = Configuration(user=username, password=password)
        client = Client(config=config)
        print("WEkEO client initialized successfully")
        return client
    except Exception as e:
        print(f"Failed to initialize WEkEO client: {str(e)}")
        raise

def build_query(latitude, longitude, start_time, end_time):
    """Construct WEkEO API query with bounding box and time range"""
    bbox_size = 0.1

    # Ensure coordinates are within valid bounds
    min_lat = max(-90.0, latitude - bbox_size)
    max_lat = min(90.0, latitude + bbox_size)
    min_lon = max(-180.0, longitude - bbox_size)
    max_lon = min(180.0, longitude + bbox_size)

    query = {
        "datasetId": "EO:EUM:DAT:MSG:CLM-IODC",
        "boundingBoxValues": [{
            "name": "bbox",
            "bbox": [min_lon, min_lat, max_lon, max_lat]
        }],
        "dateRangeSelectValues": [{
            "name": "time",
            "start": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "end": end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        }],
        "stringChoiceValues": [
            {"name": "sat", "value": "MSG1"},
            {"name": "type", "value": "MSGCLMK"}
        ],
        "itemsPerPage": 200,
        "startIndex": 0
    }

    print(f"Built query for coordinates ({latitude}, {longitude}) from {start_time} to {end_time}")
    return query

def extract_zip_files(download_folder):
    """Extract ZIP files and return list of NetCDF and GRIB files"""
    zip_files = glob.glob(os.path.join(download_folder, "*.zip"))
    processed_files = []

    for zip_file in zip_files:
        print(f"Extracting ZIP file: {os.path.basename(zip_file)}")
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                # List contents
                zip_contents = zip_ref.namelist()
                print(f"ZIP contains: {zip_contents}")

                # Extract all files
                zip_ref.extractall(download_folder)

                # Find extracted NetCDF and GRIB files
                for extracted_file in zip_contents:
                    extracted_path = os.path.join(download_folder, extracted_file)
                    if extracted_file.lower().endswith(('.nc', '.netcdf', '.grib', '.grb')):
                        processed_files.append(extracted_path)
                        print(f"Extracted data file: {extracted_file}")
                    elif os.path.isfile(extracted_path):
                        print(f"Extracted other file: {extracted_file}")

            # Remove the original ZIP file after successful extraction
            # os.remove(zip_file)
            print(f"Removed original ZIP file: {os.path.basename(zip_file)}")

        except Exception as e:
            print(f"Error extracting {zip_file}: {str(e)}")

    return processed_files


def download_files(client, matches, download_folder, limit_download_files=None):
    """Download files in batches with rate limiting to persistent folder"""
    if not matches:
        print("No matches to download")
        return []

    print(f"Starting download of {len(matches)} files (limit: {limit_download_files})")
    print(f"Files will be saved to: {download_folder}")

    downloaded_files = []

    # Handle single file download
    if limit_download_files == 1:
        try:
            print("Downloading single file...")
            matches[0].download(download_folder)

            # Check for ZIP files and extract them
            zip_files = glob.glob(os.path.join(download_folder, "*.zip"))
            if zip_files:
                print(f"Found ZIP files: {[os.path.basename(f) for f in zip_files]}")
                extracted_data_files = extract_zip_files(download_folder)
                downloaded_files.extend(extracted_data_files)

            # Also check for direct .nc or .grb files
            direct_data_files = glob.glob(os.path.join(download_folder, "*.nc")) + \
                                 glob.glob(os.path.join(download_folder, "*.grib")) + \
                                 glob.glob(os.path.join(download_folder, "*.grb"))
            downloaded_files.extend([f for f in direct_data_files if f not in downloaded_files])

            print(f"All downloaded files (including extracted): {[os.path.basename(f) for f in downloaded_files]}")
            return downloaded_files

        except Exception as e:
            print(f"Error downloading single file: {str(e)}")
            return []

    # Handle multiple file downloads (original logic for completeness)
    batch_size = min(100, limit_download_files) if limit_download_files else 100
    total_batches = (len(matches) + batch_size - 1) // batch_size

    for i in range(0, len(matches), batch_size):
        batch = matches[i:i+batch_size]
        batch_num = i//batch_size + 1
        print(f"Downloading batch {batch_num}/{total_batches} with {len(batch)} files")

        for j, match in enumerate(batch):
            try:
                print(f"Downloading file {j+1}/{len(batch)} in batch {batch_num}")
                match.download(download_folder)

                # Extract any new ZIP files and add to downloaded_files
                zip_files = glob.glob(os.path.join(download_folder, "*.zip"))
                for zip_file in zip_files:
                    if zip_file not in downloaded_files: # Avoid re-processing already added zips
                        extracted_files = extract_zip_files(download_folder)
                        downloaded_files.extend(extracted_files)

                # Add any directly downloaded .nc or .grb files
                new_data_files = glob.glob(os.path.join(download_folder, "*.nc")) + \
                                 glob.glob(os.path.join(download_folder, "*.grib")) + \
                                 glob.glob(os.path.join(download_folder, "*.grb"))
                downloaded_files.extend([f for f in new_data_files if f not in downloaded_files])

            except Exception as e:
                print(f"Error downloading file {j+1} in batch {batch_num}: {str(e)}")
                continue

        # Rate limiting between batches
        if i + batch_size < len(matches):
            print("Sleeping for 60 seconds to respect rate limits")
            time.sleep(60)

        # Check download limit
        if limit_download_files and len(downloaded_files) >= limit_download_files:
            print(f"Download limit reached: {len(downloaded_files)} files")
            break

    print(f"Total data files available for processing: {len(downloaded_files)}")
    return downloaded_files


def process_file(file_path, latitude, longitude):
    """Extract cloud status from a single NetCDF or GRIB file"""
    try:
        print(f"Processing file: {os.path.basename(file_path)}")

        ds = None
        if file_path.lower().endswith(('.nc', '.netcdf')):
            ds = xr.open_dataset(file_path)
        elif file_path.lower().endswith(('.grib', '.grb')):
            if not CFGRIB_AVAILABLE:
                print(f"Skipping GRIB file {os.path.basename(file_path)}: cfgrib not installed.")
                return None
            try:
                # Key modification: Add backend_kwargs for geostationary projection handling.
                # This instructs cfgrib to correctly interpret the grid, exposing 'y' and 'x'
                # dimensions and associated 2D 'latitude' and 'longitude' coordinates.
                # 'geostationary' is the documented GRIB gridType for MSG CLM products.
                ds = xr.open_dataset(file_path, engine="cfgrib",
                                     backend_kwargs={'filter_by_keys': {'gridType': 'geostationary'}})
                # Alternative if 'geostationary' doesn't work for specific files/versions:
                # ds = xr.open_dataset(file_path, engine="cfgrib",
                #                      backend_kwargs={'filter_by_keys': {'typeOfGrid': 'rotated_latitude_longitude'}})
            except Exception as e:
                print(f"Error opening GRIB file {os.path.basename(file_path)} with cfgrib: {str(e)}")
                return None
        else:
            print(f"Unsupported file type: {os.path.basename(file_path)}")
            return None

        with ds:
            # Debugging and verification: Print dataset structure to understand dimensions and coordinates.
            # This output is crucial for confirming the backend_kwargs' effect.
            print(f"Dataset structure for {os.path.basename(file_path)}:")
            print(ds) # This will display ds.dims, ds.coords, ds.data_vars

            # Extract timestamp
            # Added 'valid_time' which is a common time variable name in GRIB files.
            time_vars = ['time', 'Time', 't', 'nominal_product_time', 'forecast_reference_time', 'valid_time']
            time_var = next((v for v in time_vars if v in ds.variables), None)

            if not time_var:
                print(f"No time variable found in {os.path.basename(file_path)}. Available variables: {list(ds.variables.keys())}")
                return None

            try:
                time_data = ds[time_var].values
                if hasattr(time_data, '__len__') and len(time_data) > 0:
                    timestamp = pd.to_datetime(time_data).to_pydatetime()
                else:
                    timestamp = pd.to_datetime(time_data).to_pydatetime()
            except Exception as e:
                print(f"Error parsing timestamp from {time_var}: {str(e)}")
                return None

            # Find cloud mask variable
            # 'p260537' is explicitly included as it's the observed name for MSG CLM products via cfgrib.
            cloud_vars = ['cloud_mask', 'cloudmask', 'CLM', 'cloud_classification', 'cld_mask', 'p260537']
            cloud_var = next((v for v in cloud_vars if v in ds.variables), None)

            if not cloud_var:
                print(f"No cloud variable found in {os.path.basename(file_path)}. Available variables: {list(ds.variables.keys())}")
                return None

            cloud_mask = ds[cloud_var]

            # Handle time dimension if present, ensuring selection of the first time slice if multiple exist.
            time_dims = ['time', 'Time', 'valid_time', 'step'] # Added 'step' as another common time dimension in GRIB
            time_dim = next((d for d in cloud_mask.dims if d in time_dims), None)
            if time_dim and time_dim in cloud_mask.dims and len(cloud_mask[time_dim]) > 1:
                cloud_mask = cloud_mask.isel({time_dim: 0})

            # Identify coordinate dimensions for selection.
            # For geostationary data, 'latitude' and 'longitude' are expected to be 2D *coordinates*
            # of the 'y' and 'x' dimensions (e.g., latitude(y, x), longitude(y, x)).
            lat_coord_candidates = ['latitude', 'lat', 'grib_latitude']
            lon_coord_candidates = ['longitude', 'lon', 'grib_longitude']

            # Find the actual 2D latitude and longitude coordinate variable names in ds.coords
            actual_lat_coord_name = next((c for c in lat_coord_candidates if c in ds.coords and ds.coords[c].ndim == 2), None)
            actual_lon_coord_name = next((c for c in lon_coord_candidates if c in ds.coords and ds.coords[c].ndim == 2), None)

            if not actual_lat_coord_name or not actual_lon_coord_name:
                print(f"2D latitude/longitude coordinate variables not found or not correctly structured in {os.path.basename(file_path)}. Available coords: {list(ds.coords.keys())}")
                print("Expected 2D coordinates (e.g., latitude(y, x), longitude(y, x)) for geostationary data.")
                return None

            # Extract cloud value at specified coordinates using the 2D coordinates.
            # xarray.sel with method="nearest" handles the lookup on 2D coordinates.
            try:
                cloud_value = cloud_mask.sel(
                    **{actual_lat_coord_name: latitude, actual_lon_coord_name: longitude},
                    method="nearest"
                ).item()
            except Exception as e:
                print(f"Error extracting cloud value at ({latitude}, {longitude}): {str(e)}")
                print(f"Cloud mask dims: {cloud_mask.dims}, Cloud mask coords: {cloud_mask.coords}")
                return None

            # Map cloud value to status based on EUMETSAT MSG CLM Product Guide
            if cloud_value in [0, 1]:  # 0: Clear over water, 1: Clear over land
                status = 'clear'
            elif cloud_value == 2:      # 2: Cloud
                status = 'cloudy'
            elif cloud_value == 3:      # 3: No data
                status = 'not processed'
            else:
                status = f'unknown ({cloud_value})'

            result = {
                'datetime': timestamp,
                'cloud_status': status,
                'cloud_value': cloud_value,
                'file': os.path.basename(file_path),
                'file_path': file_path
            }

            print(f"Processed {os.path.basename(file_path)}: {status} (value: {cloud_value})")
            return result

    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {str(e)}")
        return None

def get_cloud_cover(latitude, longitude, start_time, end_time, username, password,
                     limit_download_files=None, download_folder=None):
    """Main function to retrieve cloud cover data"""

    # Input validation
    if not (-90 <= latitude <= 90):
        raise ValueError(f"Latitude must be between -90 and 90, got {latitude}")
    if not (-180 <= longitude <= 180):
        raise ValueError(f"Longitude must be between -180 and 180, got {longitude}")

    # Convert and validate input dates
    try:
        if isinstance(start_time, str):
            start_time = datetime.strptime(start_time, "%Y-%m-%d")
        if isinstance(end_time, str):
            end_time = datetime.strptime(end_time, "%Y-%m-%d")

        # Extend end time to end of day
        end_time = end_time + timedelta(days=1) - timedelta(seconds=1)

        if start_time >= end_time:
            raise ValueError("Start time must be before end time")

    except ValueError as e:
        print(f"Date parsing error: {str(e)}")
        raise

    # Create download folder if not provided
    if download_folder is None:
        download_folder = create_download_folder()
    else:
        os.makedirs(download_folder, exist_ok=True)
        print(f"Using provided download folder: {download_folder}")

    # Setup client and query
    try:
        client = setup_wekeo_client(username, password)
        query = build_query(latitude, longitude, start_time, end_time)

        # Search for matching files
        print("Searching for matching files...")
        matches = client.search(query)
        print(f"Found {len(matches)} matching files")

        if not matches:
            print("No data files found for the specified criteria")
            return pd.DataFrame(columns=['datetime', 'cloud_status', 'cloud_value', 'file', 'file_path'])

    except Exception as e:
        print(f"Error during search: {str(e)}")
        raise

    # Download and process files
    results = []
    try:
        files = download_files(client, matches, download_folder, limit_download_files)

        if not files:
            print("No files were successfully downloaded")
            return pd.DataFrame(columns=['datetime', 'cloud_status', 'cloud_value', 'file', 'file_path'])

        print(f"Processing {len(files)} downloaded files")
        for i, file_path in enumerate(files, 1):
            print(f"Processing file {i}/{len(files)}: {os.path.basename(file_path)}")
            result = process_file(file_path, latitude, longitude)
            if result:
                results.append(result)
            else:
                print(f"Failed to process file {i}/{len(files)}")

    except Exception as e:
        print(f"Error during file processing: {str(e)}")
        raise

    # Create and return DataFrame
    df = pd.DataFrame(results)
    if not df.empty:
        df = df.sort_values('datetime').reset_index(drop=True)
        print(f"Successfully processed {len(df)} records")
        print(f"Files are permanently stored in: {download_folder}")
    else:
        print("No valid results obtained")

    return df




In [ ]:
def main():
    """Main execution function with example usage"""

    # Check if HDA client is available
    if not HDA_AVAILABLE:
        print("Error: WEkEO HDA client not installed.")
        print("Please install it using: pip install wekeo-hda")
        return

    # Check if cfgrib is available
    if not CFGRIB_AVAILABLE:
        print("Error: cfgrib not installed. GRIB file processing will be skipped.")
        print("Please install it using: pip install cfgrib")
        # You might want to exit here or handle gracefully depending on expected file types
        # return

    # Example usage with WEkEO credentials
    username = 'chinso'
    password = 'WhctKLbfHpRVt'

    # Coordinates near Dubai
    latitude = 24.745
    longitude = 55.422

    # Date range
    start_date = '2019-02-01'
    end_date = '2019-02-02'

    # Optional: specify custom download folder
    # download_folder = "my_custom_folder"  # Uncomment to use custom folder
    download_folder = None  # Will create timestamped folder automatically

    try:
        print(f"Retrieving cloud cover data for coordinates ({latitude}, {longitude})")
        print(f"Date range: {start_date} to {end_date}")
        print("-" * 50)

        # Get cloud cover data
        cloud_cover_df = get_cloud_cover(
            latitude,
            longitude,
            start_date,
            end_date,
            username,
            password,
            limit_download_files=1,
            download_folder=download_folder
        )

        # Display results
        if not cloud_cover_df.empty:
            print("Cloud Cover Results:")
            print("=" * 50)
            # Display without file_path column for cleaner output
            display_df = cloud_cover_df.drop('file_path', axis=1)
            print(display_df.to_string(index=False))

            # Summary statistics
            print("\nSummary:")
            print("-" * 20)
            status_counts = cloud_cover_df['cloud_status'].value_counts()
            for status, count in status_counts.items():
                print(f"{status}: {count} records")

            # Show file locations
            print(f"\nDownloaded files are stored in:")
            for file_path in cloud_cover_df['file_path'].unique():
                print(f"  {file_path}")
        else:
            print("No cloud cover data retrieved.")

    except Exception as e:
        print(f"Application error: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
import os
import shutil

# Define the path to the folder you want to delete
folder_path = '/content/wekeo_downloads'

# Check if the folder exists before attempting to delete it
if os.path.exists(folder_path):
    # Use shutil.rmtree() to remove the folder and all its contents
    try:
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' and its contents deleted successfully.")
    except OSError as e:
        print(f"Error: {folder_path} : {e.strerror}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print(f"Folder '{folder_path}' does not exist.")

In [ ]:
!pip install xarray cfgrib numpy pandas

In [ ]:
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import os
import time
import glob
import sys
import zipfile
import numpy as np # Import numpy for numerical operations

# Try to import WEkEO client - handle if not installed
try:
    from hda import Client, Configuration
    HDA_AVAILABLE = True
except ImportError:
    print("WEkEO HDA client not installed. Install with: pip install wekeo-hda")
    HDA_AVAILABLE = False

# Import cfgrib for GRIB file handling
try:
    import cfgrib
    CFGRIB_AVAILABLE = True
except ImportError:
    print("cfgrib not installed. Install with: pip install cfgrib")
    CFGRIB_AVAILABLE = False


def create_download_folder(base_folder="wekeo_downloads"):
    """Create and return path to download folder"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    download_folder = os.path.join(base_folder, f"download_{timestamp}")
    os.makedirs(download_folder, exist_ok=True)
    print(f"Created download folder: {download_folder}")
    return download_folder

def setup_wekeo_client(username, password):
    """Initialize and return WEkEO client with credentials"""
    if not HDA_AVAILABLE:
        raise ImportError("WEkEO HDA client not available.")
    try:
        config = Configuration(user=username, password=password)
        client = Client(config=config)
        print("WEkEO client initialized successfully")
        return client
    except Exception as e:
        print(f"Failed to initialize WEkEO client: {str(e)}")
        raise

def build_query(latitude, longitude, start_time, end_time):
    """Construct WEkEO API query with bounding box and time range"""
    bbox_size = 0.1
    min_lat, max_lat = max(-90.0, latitude - bbox_size), min(90.0, latitude + bbox_size)
    min_lon, max_lon = max(-180.0, longitude - bbox_size), min(180.0, longitude + bbox_size)

    query = {
        "datasetId": "EO:EUM:DAT:MSG:CLM-IODC",
        "boundingBoxValues": [{"name": "bbox", "bbox": [min_lon, min_lat, max_lon, max_lat]}],
        "dateRangeSelectValues": [{
            "name": "time",
            "start": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "end": end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        }],
        "stringChoiceValues": [
            {"name": "sat", "value": "MSG1"},
            {"name": "type", "value": "MSGCLMK"}
        ],
        "itemsPerPage": 200,
        "startIndex": 0
    }
    print(f"Built query for coordinates ({latitude}, {longitude}) from {start_time} to {end_time}")
    return query

def extract_zip_files(download_folder):
    """Extract ZIP files and return list of NetCDF and GRIB files"""
    zip_files = glob.glob(os.path.join(download_folder, "*.zip"))
    processed_files = []

    if not zip_files:
        print("No ZIP files found to extract.")
        return []

    print(f"Found ZIP files: {[os.path.basename(f) for f in zip_files]}")

    for zip_file in zip_files:
        print(f"Extracting ZIP file: {os.path.basename(zip_file)}")
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_contents = zip_ref.namelist()
                print(f"ZIP contains: {zip_contents}")
                zip_ref.extractall(download_folder)

                for extracted_file in zip_contents:
                    extracted_path = os.path.join(download_folder, extracted_file)
                    if extracted_file.lower().endswith(('.nc', '.netcdf', '.grib', '.grb')):
                        processed_files.append(extracted_path)
                        print(f"Extracted data file: {extracted_file}")
                    elif os.path.isfile(extracted_path):
                        print(f"Extracted other file: {extracted_file}")
        except Exception as e:
            print(f"Error extracting {zip_file}: {str(e)}")
    return processed_files

def download_files(client, matches, download_folder, limit_download_files=None):
    """Download files using match.download and find them afterwards."""
    if not matches:
        print("No matches to download")
        return []

    # Apply limit if specified
    if limit_download_files is not None and len(matches) > limit_download_files:
        print(f"Limiting download from {len(matches)} to {limit_download_files} files.")
        matches_to_download = matches[:limit_download_files]
    else:
        matches_to_download = matches

    print(f"Starting download of {len(matches_to_download)} files (Total found: {len(matches)})")
    print(f"Files will be saved to: {download_folder}")

    batch_size = 100
    total_batches = (len(matches_to_download) + batch_size - 1) // batch_size

    for i in range(0, len(matches_to_download), batch_size):
        batch = matches_to_download[i:i + batch_size]
        batch_num = i // batch_size + 1
        print(f"Downloading batch {batch_num}/{total_batches} with {len(batch)} files")

        for j, match in enumerate(batch):
            try:
                print(f"Downloading file {j + 1}/{len(batch)} in batch {batch_num}...")
                # --- CORRECTED LINE ---
                match.download(download_folder)
                # --- END CORRECTION ---
                # Try to get a title, otherwise generic message
                title = getattr(match, 'title', f'file {j+1}')
                print(f"  ...Downloaded {os.path.basename(title)}")

            except Exception as e:
                print(f"Error downloading file {j + 1} in batch {batch_num}: {str(e)}")
                continue # Continue to the next file

        # Rate limiting between batches (if more than one batch)
        if i + batch_size < len(matches_to_download):
            print("Sleeping for 10 seconds between batches...")
            time.sleep(10)

    # --- Process Downloaded Files (Extract ZIPs and Collect Data Files) ---
    print("\nDownload complete. Processing downloaded folder...")

    # Extract any ZIP files found in the download folder
    extracted_data_files = extract_zip_files(download_folder)

    # Find all data files (extracted + any non-ZIP downloads)
    all_data_files = list(extracted_data_files) # Start with extracted

    # Add any directly downloaded data files (non-ZIP)
    data_extensions = ["*.nc", "*.netcdf", "*.grib", "*.grb"]
    for ext in data_extensions:
        direct_files = glob.glob(os.path.join(download_folder, ext))
        # Add only if not already in the list (from extraction)
        for f in direct_files:
            if f not in all_data_files:
                all_data_files.append(f)

    # Remove duplicates if any (just in case)
    all_data_files = list(set(all_data_files))

    print(f"All available data files: {[os.path.basename(f) for f in all_data_files]}")
    return all_data_files

def process_file(file_path, latitude, longitude):
    """Extract cloud status from a single NetCDF or GRIB file"""
    try:
        print(f"Processing file: {os.path.basename(file_path)}")
        ds = None
        if file_path.lower().endswith(('.grib', '.grb')):
            if not CFGRIB_AVAILABLE: return None
            try:
                ds = xr.open_dataset(file_path, engine='cfgrib',
                                     backend_kwargs={'filter_by_keys': {'discipline': 3, 'parameterCategory': 0, 'parameterNumber': 7},
                                                     'indexpath': ''})
            except Exception as e:
                print(f"Error opening GRIB file {os.path.basename(file_path)} with keys: {str(e)}. Trying without...")
                try: ds = xr.open_dataset(file_path, engine='cfgrib', backend_kwargs={'indexpath': ''})
                except Exception as e2: print(f"Fallback GRIB open failed: {str(e2)}"); return None
        elif file_path.lower().endswith(('.nc', '.netcdf')):
            ds = xr.open_dataset(file_path)
        else: print(f"Unsupported file type: {os.path.basename(file_path)}"); return None

        if ds is None: return None

        with ds:
            # print(f"Dataset structure:\n{ds}") # Keep this commented unless debugging
            time_var = next((v for v in ['time', 'Time', 't', 'nominal_product_time', 'valid_time'] if v in ds.variables or v in ds.coords), None)
            if not time_var: print(f"No time variable found in {os.path.basename(file_path)}"); return None
            timestamp = pd.to_datetime(ds[time_var].values.item() if hasattr(ds[time_var].values, 'item') else ds[time_var].values[0]).to_pydatetime()

            cloud_var_name = next((v for v in ['cloud_mask', 'p260537', 'CLM'] if v in ds.data_vars), None)
            if not cloud_var_name: print(f"No cloud variable found in {os.path.basename(file_path)}"); return None
            cloud_mask = ds[cloud_var_name]

            if 'latitude' in ds.coords and 'longitude' in ds.coords:
                try:
                    if 'values' in cloud_mask.dims:
                        lats, lons = cloud_mask.latitude.values, cloud_mask.longitude.values
                        distances = np.sqrt((lats - latitude)**2 + (lons - longitude)**2)
                        cloud_value = cloud_mask.isel(values=np.argmin(distances)).item()
                    else:
                        cloud_value = cloud_mask.sel(latitude=latitude, longitude=longitude, method='nearest').item()
                except Exception as e: print(f"Error extracting cloud value: {str(e)}"); return None
            else: print(f"Lat/Lon coords not found in {os.path.basename(file_path)}"); return None

            status_map = {0: 'clear (water)', 1: 'clear (land)', 2: 'cloudy', 3: 'not processed'}
            status = status_map.get(cloud_value, f'unknown ({cloud_value})')
            clear_status = 'clear' if status.startswith('clear') else status

            result = {'datetime': timestamp, 'cloud_status': clear_status, 'cloud_value': cloud_value, 'file': os.path.basename(file_path), 'file_path': file_path}
            print(f"Processed {os.path.basename(file_path)}: {status} (value: {cloud_value})")
            return result
    except Exception as e:
        print(f"Major error processing {os.path.basename(file_path)}: {str(e)}")
        return None

def get_cloud_cover(latitude, longitude, start_time_str, end_time_str, username, password,
                      limit_download_files=None, download_folder=None):
    """Main function to retrieve cloud cover data"""
    if not (-90 <= latitude <= 90 and -180 <= longitude <= 180): raise ValueError("Invalid coordinates")

    try:
        start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
        end_time = datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S")
        if start_time >= end_time: raise ValueError("Start time must be before end time")
    except ValueError as e: print(f"Date/Time parsing error: {e}. Use 'YYYY-MM-DD HH:MM:SS'."); raise

    download_folder = download_folder or create_download_folder()
    os.makedirs(download_folder, exist_ok=True)

    try:
        client = setup_wekeo_client(username, password)
        query = build_query(latitude, longitude, start_time, end_time)
        print("Searching for matching files...")
        matches = client.search(query)
        print(f"Found {len(matches)} matching files")
        if not matches: return pd.DataFrame()
    except Exception as e: print(f"Error during search: {e}"); raise

    results = []
    try:
        files = download_files(client, matches, download_folder, limit_download_files)
        if not files: print("No files were downloaded/extracted"); return pd.DataFrame()

        print(f"\nProcessing {len(files)} data files...")
        for i, file_path in enumerate(files, 1):
            result = process_file(file_path, latitude, longitude)
            if result: results.append(result)
            else: print(f"Failed to process {os.path.basename(file_path)}")
    except Exception as e: print(f"Error during processing: {e}"); raise

    df = pd.DataFrame(results)
    if not df.empty:
        df = df.sort_values('datetime').reset_index(drop=True)
        print(f"\nSuccessfully processed {len(df)} records.")
        print(f"Files stored in: {download_folder}")
    else: print("\nNo valid results obtained.")
    return df

def main():
    """Main execution function"""
    if not HDA_AVAILABLE: print("Error: WEkEO HDA client not installed."); return
    if not CFGRIB_AVAILABLE: print("Warning: cfgrib not installed. GRIB processing may fail.");

    username = 'chinso' # Replace with your username
    password = 'WhctKLbfHpRVt' # Replace with your password

    latitude = 24.745
    longitude = 55.422

    # Use the 2019 date range from your output
    start_time_str = '2025-02-02 12:00:00'
    end_time_str = '2025-02-02 17:00:00'

    download_limit = 5 # Keep the limit at 5 as in your example

    try:
        print(f"Retrieving cloud cover data for ({latitude}, {longitude})")
        print(f"Time range: {start_time_str} to {end_time_str}")
        print("-" * 50)
        cloud_cover_df = get_cloud_cover(latitude, longitude, start_time_str, end_time_str,
                                           username, password, limit_download_files=download_limit)

        if not cloud_cover_df.empty:
            print("\nCloud Cover Results:")
            print("=" * 50)
            print(cloud_cover_df.drop('file_path', axis=1).to_string(index=False))
            print("\nSummary:")
            print("-" * 20)
            print(cloud_cover_df['cloud_status'].value_counts())
            print(f"\nFiles are in: {os.path.dirname(cloud_cover_df['file_path'].iloc[0])}")
        else:
            print("No cloud cover data retrieved.")
    except Exception as e:
        print(f"\nApplication error: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import os
import time
import glob
import sys
import zipfile
import numpy as np # Import numpy for numerical operations

# Try to import WEkEO client - handle if not installed
try:
    from hda import Client, Configuration
    HDA_AVAILABLE = True
except ImportError:
    print("WEkEO HDA client not installed. Install with: pip install wekeo-hda")
    HDA_AVAILABLE = False

# Import cfgrib for GRIB file handling
try:
    import cfgrib
    CFGRIB_AVAILABLE = True
except ImportError:
    print("cfgrib not installed. Install with: pip install cfgrib")
    CFGRIB_AVAILABLE = False


def create_download_folder(base_folder="wekeo_downloads"):
    """Create and return path to download folder"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    download_folder = os.path.join(base_folder, f"download_{timestamp}")
    os.makedirs(download_folder, exist_ok=True)
    print(f"Created download folder: {download_folder}")
    return download_folder

def setup_wekeo_client(username, password):
    """Initialize and return WEkEO client with credentials"""
    if not HDA_AVAILABLE:
        raise ImportError("WEkEO HDA client not available.")
    try:
        config = Configuration(user=username, password=password)
        client = Client(config=config)
        print("WEkEO client initialized successfully")
        return client
    except Exception as e:
        print(f"Failed to initialize WEkEO client: {str(e)}")
        raise

def build_query(bbox_coords, start_time, end_time):
    """Construct WEkEO API query with bounding box and time range"""
    query = {
        "datasetId": "EO:EUM:DAT:MSG:CLM-IODC",
        "boundingBoxValues": [{"name": "bbox", "bbox": bbox_coords}],
        "dateRangeSelectValues": [{
            "name": "time",
            "start": start_time.isoformat(timespec='milliseconds') + 'Z',
            "end": end_time.isoformat(timespec='milliseconds') + 'Z'
        }],
        "stringChoiceValues":[],
        "itemsPerPage": 200,
        "startIndex": 0
    }
    print(f"Built query for bbox {bbox_coords} from {start_time} to {end_time}")
    return query

def extract_zip_files(download_folder):
    """Extract ZIP files and return list of NetCDF and GRIB files"""
    zip_files = glob.glob(os.path.join(download_folder, "*.zip"))
    processed_files = []

    if not zip_files:
        print("No ZIP files found to extract.")
        return

    print(f"Found ZIP files: {[os.path.basename(f) for f in zip_files]}")

    for zip_file in zip_files:
        print(f"Extracting ZIP file: {os.path.basename(zip_file)}")
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_contents = zip_ref.namelist()
                print(f"ZIP contains: {zip_contents}")
                zip_ref.extractall(download_folder)

                for extracted_file in zip_contents:
                    extracted_path = os.path.join(download_folder, extracted_file)
                    if extracted_file.lower().endswith(('.nc', '.netcdf', '.grib', '.grb')):
                        processed_files.append(extracted_path)
                        print(f"Extracted data file: {extracted_file}")
                    elif os.path.isfile(extracted_path):
                        print(f"Extracted other file: {extracted_file}")
        except Exception as e:
            print(f"Error extracting {zip_file}: {str(e)}")
    return processed_files

def download_files(client, matches, download_folder, limit_download_files=None):
    """Download files using match.download and find them afterwards."""
    if not matches:
        print("No matches to download")
        return

    # Apply limit if specified
    if limit_download_files is not None and len(matches) > limit_download_files:
        print(f"Limiting download from {len(matches)} to {limit_download_files} files.")
        matches_to_download = matches[:limit_download_files]
    else:
        matches_to_download = matches

    print(f"Starting download of {len(matches_to_download)} files (Total found: {len(matches)})")
    print(f"Files will be saved to: {download_folder}")

    batch_size = 100
    total_batches = (len(matches_to_download) + batch_size - 1) // batch_size

    for i in range(0, len(matches_to_download), batch_size):
        batch = matches_to_download[i:i + batch_size]
        batch_num = i // batch_size + 1
        print(f"Downloading batch {batch_num}/{total_batches} with {len(batch)} files")

        for j, match in enumerate(batch):
            try:
                print(f"Downloading file {j + 1}/{len(batch)} in batch {batch_num}...")
                # --- CORRECTED LINE ---
                match.download(download_folder)
                # --- END CORRECTION ---
                # Try to get a title, otherwise generic message
                title = getattr(match, 'title', f'file {j+1}')
                print(f"  ...Downloaded {os.path.basename(title)}")

            except Exception as e:
                print(f"Error downloading file {j + 1} in batch {batch_num}: {str(e)}")
                continue # Continue to the next file

        # Rate limiting between batches (if more than one batch)
        if i + batch_size < len(matches_to_download):
            print("Sleeping for 10 seconds between batches...")
            time.sleep(10)

    # --- Process Downloaded Files (Extract ZIPs and Collect Data Files) ---
    print("\nDownload complete. Processing downloaded folder...")

    # Extract any ZIP files found in the download folder
    extracted_data_files = extract_zip_files(download_folder)

    # Find all data files (extracted + any non-ZIP downloads)
    all_data_files = list(extracted_data_files) # Start with extracted

    # Add any directly downloaded data files (non-ZIP)
    data_extensions = ["*.nc", "*.netcdf", "*.grib", "*.grb"]
    for ext in data_extensions:
        direct_files = glob.glob(os.path.join(download_folder, ext))
        # Add only if not already in the list (from extraction)
        for f in direct_files:
            if f not in all_data_files:
                all_data_files.append(f)

    # Remove duplicates if any (just in case)
    all_data_files = list(set(all_data_files))

    print(f"All available data files: {[os.path.basename(f) for f in all_data_files]}")
    return all_data_files

def process_file(file_path, latitude, longitude):
    """Extract cloud status from a single NetCDF or GRIB file"""
    try:
        print(f"Processing file: {os.path.basename(file_path)}")
        ds = None
        if file_path.lower().endswith(('.grib', '.grb')):
            if not CFGRIB_AVAILABLE: return None
            try:
                ds = xr.open_dataset(file_path, engine='cfgrib',
                                     backend_kwargs={'filter_by_keys': {'discipline': 3, 'parameterCategory': 0, 'parameterNumber': 7},
                                                     'indexpath': ''})
            except Exception as e:
                print(f"Error opening GRIB file {os.path.basename(file_path)} with keys: {str(e)}. Trying without...")
                try: ds = xr.open_dataset(file_path, engine='cfgrib', backend_kwargs={'indexpath': ''})
                except Exception as e2: print(f"Fallback GRIB open failed: {str(e2)}"); return None
        elif file_path.lower().endswith(('.nc', '.netcdf')):
            ds = xr.open_dataset(file_path)
        else: print(f"Unsupported file type: {os.path.basename(file_path)}"); return None

        if ds is None: return None

        with ds:
            # print(f"Dataset structure:\n{ds}") # Keep this commented unless debugging
            time_var = next((v for v in ds.variables if v in ds.coords or v in ds.variables), None)
            if not time_var: print(f"No time variable found in {os.path.basename(file_path)}"); return None
            timestamp = pd.to_datetime(ds[time_var].values.item() if hasattr(ds[time_var].values, 'item') else ds[time_var].values).to_pydatetime()

            cloud_var_name = next((v for v in ['cloud_mask', 'p260537', 'CLM'] if v in ds.data_vars), None)
            if not cloud_var_name: print(f"No cloud variable found in {os.path.basename(file_path)}"); return None
            cloud_mask = ds[cloud_var_name]

            if 'latitude' in ds.coords and 'longitude' in ds.coords:
                try:
                    if 'values' in cloud_mask.dims:
                        lats, lons = cloud_mask.latitude.values, cloud_mask.longitude.values
                        distances = np.sqrt((lats - latitude)**2 + (lons - longitude)**2)
                        cloud_value = cloud_mask.isel(values=np.argmin(distances)).item()
                    else:
                        cloud_value = cloud_mask.sel(latitude=latitude, longitude=longitude, method='nearest').item()
                except Exception as e: print(f"Error extracting cloud value: {str(e)}"); return None
            else: print(f"Lat/Lon coords not found in {os.path.basename(file_path)}"); return None

            status_map = {0: 'clear (water)', 1: 'clear (land)', 2: 'cloudy', 3: 'not processed'}
            status = status_map.get(cloud_value, f'unknown ({cloud_value})')
            clear_status = 'clear' if status.startswith('clear') else status

            result = {'datetime': timestamp, 'cloud_status': clear_status, 'cloud_value': cloud_value, 'file': os.path.basename(file_path), 'file_path': file_path}
            print(f"Processed {os.path.basename(file_path)}: {status} (value: {cloud_value})")
            return result
    except Exception as e:
        print(f"Major error processing {os.path.basename(file_path)}: {str(e)}")
        return None

def get_cloud_cover(bbox_coords, start_time_str, end_time_str, username, password,
                     limit_download_files=None, download_folder=None):
    """Main function to retrieve cloud cover data"""
    # Validate bbox_coords format and ranges
    if not (isinstance(bbox_coords, list) and len(bbox_coords) == 4):
        raise ValueError("bbox_coords must be a list of 4 values: [min_lon, min_lat, max_lon, max_lat]")
    min_lon, min_lat, max_lon, max_lat = bbox_coords
    if not (-180 <= min_lon <= 180 and -180 <= max_lon <= 180 and
            -90 <= min_lat <= 90 and -90 <= max_lat <= 90 and
            min_lon <= max_lon and min_lat <= max_lat):
        raise ValueError("Invalid bbox_coords values or order.")

    # Calculate center lat/lon for process_file
    latitude = (min_lat + max_lat) / 2
    longitude = (min_lon + max_lon) / 2

    try:
        start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
        end_time = datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S")
        if start_time >= end_time: raise ValueError("Start time must be before end time")
    except ValueError as e: print(f"Date/Time parsing error: {e}. Use 'YYYY-MM-DD HH:MM:SS'."); raise

    download_folder = download_folder or create_download_folder()
    os.makedirs(download_folder, exist_ok=True)

    try:
        client = setup_wekeo_client(username, password)
        query = build_query(bbox_coords, start_time, end_time) # Pass bbox_coords
        print("Searching for matching files...")
        matches = client.search(query)
        print(f"Found {len(matches)} matching files")
        if not matches: return pd.DataFrame()
    except Exception as e: print(f"Error during search: {e}"); raise

    results = []
    try:
        files = download_files(client, matches, download_folder, limit_download_files)
        if not files: print("No files were downloaded/extracted"); return pd.DataFrame()

        print(f"\nProcessing {len(files)} data files...")
        for i, file_path in enumerate(files, 1):
            result = process_file(file_path, latitude, longitude) # Pass derived lat/lon
            if result: results.append(result)
            else: print(f"Failed to process {os.path.basename(file_path)}")
    except Exception as e: print(f"Error during processing: {e}"); raise

    df = pd.DataFrame(results)
    if not df.empty:
        df = df.sort_values('datetime').reset_index(drop=True)
        print(f"\nSuccessfully processed {len(df)} records.")
        print(f"Files stored in: {download_folder}")
    else: print("\nNo valid results obtained.")
    return df

def main():
    """Main execution function"""
    if not HDA_AVAILABLE: print("Error: WEkEO HDA client not installed."); return
    if not CFGRIB_AVAILABLE: print("Warning: cfgrib not installed. GRIB processing may fail.");

    username = 'chinso' # Replace with your username
    password = 'WhctKLbfHpRVt' # Replace with your password

    # Updated parameters based on the provided API request JSON
    start_time_str = '2025-02-02 00:00:00'
    end_time_str = '2025-02-03 00:00:00'
    bbox_coords = [
        55.43467643191375,
        24.72290881524073,
        55.51147607683789,
        24.78228805026869
    ]

    download_limit = None # Keep the limit at 5 as in your example

    try:
        print(f"Retrieving cloud cover data for bbox {bbox_coords}")
        print(f"Time range: {start_time_str} to {end_time_str}")
        print("-" * 50)
        cloud_cover_df = get_cloud_cover(bbox_coords, start_time_str, end_time_str,
                                         username, password, limit_download_files=download_limit)

        if not cloud_cover_df.empty:
            print("\nCloud Cover Results:")
            print("=" * 50)
            print(cloud_cover_df.drop('file_path', axis=1).to_string(index=False))
            print("\nSummary:")
            print("-" * 20)
            print(cloud_cover_df['cloud_status'].value_counts())
            print(f"\nFiles are in: {os.path.dirname(cloud_cover_df['file_path'].iloc)}")
        else:
            print("No cloud cover data retrieved.")
    except Exception as e:
        print(f"\nApplication error: {e}")

if __name__ == "__main__":
    main()

In [ ]:
"""
Script para extraer datos de cobertura de nubes de EUMETSAT MSG CLM-IODC usando WEkEO HDA
Autor: Manus
Fecha: Mayo 2025
"""

import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import os
import time
import glob
import sys
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# Importar cliente WEkEO HDA
try:
    from hda import Client, Configuration
    HDA_AVAILABLE = True
    print("Cliente WEkEO HDA importado correctamente")
except ImportError:
    print("WEkEO HDA no está instalado. Instálalo con: pip install wekeo-hda")
    print("Comando de instalación: pip install git+https://github.com/wekeo/wekeo-hda.git")
    HDA_AVAILABLE = False
    sys.exit(1)

# Importar cfgrib para manejo de archivos GRIB
try:
    import cfgrib
    CFGRIB_AVAILABLE = True
    print("cfgrib importado correctamente")
except ImportError:
    print("cfgrib no está instalado. Instálalo con: pip install cfgrib")
    CFGRIB_AVAILABLE = False
    sys.exit(1)

def create_download_folder(base_folder="wekeo_downloads"):
    """Crear y devolver la ruta a la carpeta de descarga"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    download_folder = os.path.join(base_folder, f"download_{timestamp}")
    os.makedirs(download_folder, exist_ok=True)
    print(f"Carpeta de descarga creada: {download_folder}")
    return download_folder

def setup_wekeo_client(username, password):
    """Inicializar y devolver el cliente WEkEO con credenciales"""
    if not HDA_AVAILABLE:
        raise ImportError("Cliente WEkEO HDA no disponible.")
    try:
        config = Configuration(user=username, password=password)
        client = Client(config=config)
        print("Cliente WEkEO inicializado correctamente")
        return client
    except Exception as e:
        print(f"Error al inicializar el cliente WEkEO: {str(e)}")
        raise

def build_query(bbox_coords, start_time, end_time):
    """Construir consulta de API WEkEO con bounding box y rango de tiempo"""
    query = {
        "datasetId": "EO:EUM:DAT:MSG:CLM-IODC",
        "boundingBoxValues": [{"name": "bbox", "bbox": bbox_coords}],
        "dateRangeSelectValues": [{
            "name": "time",
            "start": start_time.isoformat(timespec='milliseconds') + 'Z',
            "end": end_time.isoformat(timespec='milliseconds') + 'Z'
        }],
        "stringChoiceValues":[],
        "itemsPerPage": 200,
        "startIndex": 0
    }
    print(f"Consulta construida para bbox {bbox_coords} desde {start_time} hasta {end_time}")
    return query

def extract_zip_files(download_folder):
    """Extraer archivos ZIP y devolver lista de archivos NetCDF y GRIB"""
    zip_files = glob.glob(os.path.join(download_folder, "*.zip"))
    processed_files = []

    if not zip_files:
        print("No se encontraron archivos ZIP para extraer.")
        return []

    print(f"Archivos ZIP encontrados: {[os.path.basename(f) for f in zip_files]}")

    for zip_file in zip_files:
        print(f"Extrayendo archivo ZIP: {os.path.basename(zip_file)}")
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_contents = zip_ref.namelist()
                print(f"ZIP contiene: {zip_contents}")
                zip_ref.extractall(download_folder)

                for extracted_file in zip_contents:
                    extracted_path = os.path.join(download_folder, extracted_file)
                    if extracted_file.lower().endswith(('.nc', '.netcdf', '.grib', '.grb')):
                        processed_files.append(extracted_path)
                        print(f"Archivo de datos extraído: {extracted_file}")
                    elif os.path.isfile(extracted_path):
                        print(f"Otro archivo extraído: {extracted_file}")
        except Exception as e:
            print(f"Error al extraer {zip_file}: {str(e)}")
    return processed_files

def download_files(client, matches, download_folder, limit_download_files=None):
    """Descargar archivos usando match.download y encontrarlos después."""
    if not matches:
        print("No hay coincidencias para descargar")
        return []

    # Aplicar límite si se especifica
    if limit_download_files is not None and len(matches) > limit_download_files:
        print(f"Limitando descarga de {len(matches)} a {limit_download_files} archivos.")
        matches_to_download = matches[:limit_download_files]
    else:
        matches_to_download = matches

    print(f"Iniciando descarga de {len(matches_to_download)} archivos (Total encontrados: {len(matches)})")
    print(f"Los archivos se guardarán en: {download_folder}")

    batch_size = 100
    total_batches = (len(matches_to_download) + batch_size - 1) // batch_size

    for i in range(0, len(matches_to_download), batch_size):
        batch = matches_to_download[i:i + batch_size]
        batch_num = i // batch_size + 1
        print(f"Descargando lote {batch_num}/{total_batches} con {len(batch)} archivos")

        for j, match in enumerate(batch):
            try:
                print(f"Descargando archivo {j + 1}/{len(batch)} en lote {batch_num}...")
                match.download(download_folder)
                # Intentar obtener un título, de lo contrario mensaje genérico
                title = getattr(match, 'title', f'archivo {j+1}')
                print(f"  ...Descargado {os.path.basename(title)}")

            except Exception as e:
                print(f"Error al descargar archivo {j + 1} en lote {batch_num}: {str(e)}")
                continue # Continuar con el siguiente archivo

        # Limitación de velocidad entre lotes (si hay más de un lote)
        if i + batch_size < len(matches_to_download):
            print("Esperando 10 segundos entre lotes...")
            time.sleep(10)

    # --- Procesar archivos descargados (extraer ZIPs y recopilar archivos de datos) ---
    print("\nDescarga completa. Procesando carpeta de descarga...")

    # Extraer cualquier archivo ZIP encontrado en la carpeta de descarga
    extracted_data_files = extract_zip_files(download_folder)

    # Encontrar todos los archivos de datos (extraídos + cualquier descarga no ZIP)
    all_data_files = list(extracted_data_files) # Comenzar con los extraídos

    # Agregar cualquier archivo de datos descargado directamente (no ZIP)
    data_extensions = ["*.nc", "*.netcdf", "*.grib", "*.grb"]
    for ext in data_extensions:
        direct_files = glob.glob(os.path.join(download_folder, ext))
        # Agregar solo si no está ya en la lista (de la extracción)
        for f in direct_files:
            if f not in all_data_files:
                all_data_files.append(f)

    # Eliminar duplicados si los hay (por si acaso)
    all_data_files = list(set(all_data_files))

    print(f"Todos los archivos de datos disponibles: {[os.path.basename(f) for f in all_data_files]}")
    return all_data_files

def process_file(file_path, latitude, longitude):
    """Extraer estado de nube de un solo archivo NetCDF o GRIB"""
    try:
        print(f"Procesando archivo: {os.path.basename(file_path)}")
        ds = None
        if file_path.lower().endswith(('.grib', '.grb')):
            if not CFGRIB_AVAILABLE: return None
            try:
                ds = xr.open_dataset(file_path, engine='cfgrib',
                                     backend_kwargs={'filter_by_keys': {'discipline': 3, 'parameterCategory': 0, 'parameterNumber': 7},
                                                     'indexpath': ''})
            except Exception as e:
                print(f"Error al abrir archivo GRIB {os.path.basename(file_path)} con claves: {str(e)}. Intentando sin...")
                try: ds = xr.open_dataset(file_path, engine='cfgrib', backend_kwargs={'indexpath': ''})
                except Exception as e2: print(f"Apertura GRIB alternativa falló: {str(e2)}"); return None
        elif file_path.lower().endswith(('.nc', '.netcdf')):
            ds = xr.open_dataset(file_path)
        else: print(f"Tipo de archivo no soportado: {os.path.basename(file_path)}"); return None

        if ds is None: return None

        with ds:
            # Imprimir estructura del dataset para depuración
            print(f"Estructura del dataset:\n{ds}")

            time_var = next((v for v in ds.variables if v in ds.coords or v in ds.variables), None)
            if not time_var: print(f"No se encontró variable de tiempo en {os.path.basename(file_path)}"); return None
            timestamp = pd.to_datetime(ds[time_var].values.item() if hasattr(ds[time_var].values, 'item') else ds[time_var].values).to_pydatetime()

            cloud_var_name = next((v for v in ['cloud_mask', 'p260537', 'CLM'] if v in ds.data_vars), None)
            if not cloud_var_name:
                print(f"No se encontró variable de nube en {os.path.basename(file_path)}")
                # Mostrar todas las variables disponibles
                print(f"Variables disponibles: {list(ds.data_vars.keys())}")
                return None

            cloud_mask = ds[cloud_var_name]

            if 'latitude' in ds.coords and 'longitude' in ds.coords:
                try:
                    if 'values' in cloud_mask.dims:
                        lats, lons = cloud_mask.latitude.values, cloud_mask.longitude.values
                        distances = np.sqrt((lats - latitude)**2 + (lons - longitude)**2)
                        cloud_value = cloud_mask.isel(values=np.argmin(distances)).item()
                    else:
                        cloud_value = cloud_mask.sel(latitude=latitude, longitude=longitude, method='nearest').item()
                except Exception as e: print(f"Error al extraer valor de nube: {str(e)}"); return None
            else: print(f"Coords lat/lon no encontradas en {os.path.basename(file_path)}"); return None

            status_map = {0: 'clear (water)', 1: 'clear (land)', 2: 'cloudy', 3: 'not processed'}
            status = status_map.get(cloud_value, f'unknown ({cloud_value})')
            clear_status = 'clear' if status.startswith('clear') else status

            result = {'datetime': timestamp, 'cloud_status': clear_status, 'cloud_value': cloud_value, 'file': os.path.basename(file_path), 'file_path': file_path}
            print(f"Procesado {os.path.basename(file_path)}: {status} (valor: {cloud_value})")
            return result
    except Exception as e:
        print(f"Error grave procesando {os.path.basename(file_path)}: {str(e)}")
        return None

def get_cloud_cover(bbox_coords, start_time_str, end_time_str, username, password,
                     limit_download_files=None, download_folder=None):
    """Función principal para recuperar datos de cobertura de nubes"""
    # Validar formato y rangos de bbox_coords
    if not (isinstance(bbox_coords, list) and len(bbox_coords) == 4):
        raise ValueError("bbox_coords debe ser una lista de 4 valores: [min_lon, min_lat, max_lon, max_lat]")
    min_lon, min_lat, max_lon, max_lat = bbox_coords
    if not (-180 <= min_lon <= 180 and -180 <= max_lon <= 180 and
            -90 <= min_lat <= 90 and -90 <= max_lat <= 90 and
            min_lon <= max_lon and min_lat <= max_lat):
        raise ValueError("Valores o orden de bbox_coords inválidos.")

    # Calcular centro lat/lon para process_file
    latitude = (min_lat + max_lat) / 2
    longitude = (min_lon + max_lon) / 2

    try:
        start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
        end_time = datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S")
        if start_time >= end_time: raise ValueError("La hora de inicio debe ser anterior a la hora de fin")
    except ValueError as e: print(f"Error de análisis de fecha/hora: {e}. Use 'YYYY-MM-DD HH:MM:SS'."); raise

    download_folder = download_folder or create_download_folder()
    os.makedirs(download_folder, exist_ok=True)

    try:
        client = setup_wekeo_client(username, password)
        query = build_query(bbox_coords, start_time, end_time) # Pasar bbox_coords
        print("Buscando archivos coincidentes...")
        matches = client.search(query)
        print(f"Encontrados {len(matches)} archivos coincidentes")
        if not matches: return pd.DataFrame()
    except Exception as e: print(f"Error durante la búsqueda: {e}"); raise

    results = []
    try:
        files = download_files(client, matches, download_folder, limit_download_files)
        if not files: print("No se descargaron/extrajeron archivos"); return pd.DataFrame()

        print(f"\nProcesando {len(files)} archivos de datos...")
        for i, file_path in enumerate(files, 1):
            result = process_file(file_path, latitude, longitude) # Pasar lat/lon derivados
            if result: results.append(result)
            else: print(f"No se pudo procesar {os.path.basename(file_path)}")
    except Exception as e: print(f"Error durante el procesamiento: {e}"); raise

    df = pd.DataFrame(results)
    if not df.empty:
        df = df.sort_values('datetime').reset_index(drop=True)
        print(f"\nProcesados con éxito {len(df)} registros.")
        print(f"Archivos almacenados en: {download_folder}")
    else: print("\nNo se obtuvieron resultados válidos.")
    return df

def calculate_daylight_hours(latitude, longitude, date):
    """Calcular horas aproximadas de amanecer y atardecer para una ubicación y fecha dadas"""
    # Este es un cálculo simplificado para demostración
    # Para cálculos más precisos, considere usar bibliotecas como astral o ephem

    # Convertir fecha a día del año
    day_of_year = date.timetuple().tm_yday

    # Calcular ángulo de declinación
    declination = 23.45 * np.sin(np.radians(360/365 * (day_of_year - 81)))

    # Calcular duración del día en horas
    day_length = 24 - (24/np.pi) * np.arccos(
        (np.sin(np.radians(-0.83)) + np.sin(np.radians(latitude)) *
         np.sin(np.radians(declination))) /
        (np.cos(np.radians(latitude)) * np.cos(np.radians(declination)))
    )

    # Calcular horas de amanecer y atardecer
    noon = datetime(date.year, date.month, date.day, 12, 0)
    sunrise = noon - timedelta(hours=day_length/2)
    sunset = noon + timedelta(hours=day_length/2)

    return sunrise, sunset

def analyze_cloud_cover_by_day(cloud_cover_df, bbox_coords):
    """Analizar datos de cobertura de nubes por día, enfocándose en períodos de 15 minutos durante la luz del día"""
    if cloud_cover_df.empty:
        return pd.DataFrame()

    # Calcular centro del bbox para cálculos de luz diurna
    latitude = (bbox_coords[1] + bbox_coords[3]) / 2
    longitude = (bbox_coords[0] + bbox_coords[2]) / 2

    # Agrupar datos por fecha
    cloud_cover_df['date'] = cloud_cover_df['datetime'].dt.date

    # Inicializar resultados
    daily_results = []

    # Procesar cada día
    for date, group in cloud_cover_df.groupby('date'):
        date_obj = datetime.combine(date, datetime.min.time())

        # Calcular amanecer y atardecer para esta fecha
        sunrise, sunset = calculate_daylight_hours(latitude, longitude, date_obj)

        # Filtrar datos para horas diurnas
        daylight_data = group[(group['datetime'] >= sunrise) & (group['datetime'] <= sunset)]

        if daylight_data.empty:
            print(f"No hay datos diurnos disponibles para {date}")
            continue

        # Remuestrear a intervalos de 15 minutos
        # Primero, establecer datetime como índice
        daylight_data = daylight_data.set_index('datetime')

        # Crear bins de 15 minutos desde el amanecer hasta el atardecer
        time_bins = pd.date_range(start=sunrise, end=sunset, freq='15min')

        # Inicializar contadores
        total_periods = len(time_bins) - 1
        cloudy_periods = 0
        clear_periods = 0
        alternating_count = 0
        previous_status = None

        # Analizar cada período de 15 minutos
        for i in range(len(time_bins) - 1):
            period_start = time_bins[i]
            period_end = time_bins[i+1]

            # Obtener puntos de datos en este período
            period_data = daylight_data[(daylight_data.index >= period_start) &
                                        (daylight_data.index < period_end)]

            if period_data.empty:
                # No hay datos para este período
                continue

            # Determinar estado dominante en este período
            status_counts = period_data['cloud_status'].value_counts()
            if status_counts.empty:
                continue

            current_status = status_counts.index[0]

            # Contar períodos nublados y despejados
            if current_status == 'cloudy':
                cloudy_periods += 1
            elif current_status == 'clear':
                clear_periods += 1

            # Verificar patrón alternante
            if previous_status is not None and current_status != previous_status:
                alternating_count += 1

            previous_status = current_status

        # Calcular porcentajes
        if total_periods > 0:
            cloudy_percentage = (cloudy_periods / total_periods) * 100
            clear_percentage = (clear_periods / total_periods) * 100
            alternating_percentage = (alternating_count / (total_periods - 1)) * 100 if total_periods > 1 else 0
        else:
            cloudy_percentage = clear_percentage = alternating_percentage = 0

        # Determinar clasificación diaria basada en predominancia
        is_cloudy = cloudy_percentage > clear_percentage

        # Agregar a resultados
        daily_results.append({
            'date': date,
            'sunrise': sunrise,
            'sunset': sunset,
            'total_15min_periods': total_periods,
            'cloudy_periods': cloudy_periods,
            'clear_periods': clear_periods,
            'alternating_count': alternating_count,
            'cloudy_percentage': cloudy_percentage,
            'clear_percentage': clear_percentage,
            'alternating_percentage': alternating_percentage,
            'is_cloudy': is_cloudy
        })

    return pd.DataFrame(daily_results)

def plot_cloud_cover_timeline(cloud_cover_df, output_file):
    """Crear una visualización de línea de tiempo de datos de cobertura de nubes"""
    if cloud_cover_df.empty:
        print("No hay datos para graficar")
        return

    # Crear una nueva figura
    plt.figure(figsize=(12, 6))

    # Mapear estado de nube a valores numéricos para graficar
    status_map = {'clear': 0, 'cloudy': 1, 'not processed': -1}
    cloud_cover_df['status_value'] = cloud_cover_df['cloud_status'].map(lambda x: status_map.get(x, -1))

    # Graficar los datos
    plt.scatter(cloud_cover_df['datetime'], cloud_cover_df['status_value'],
                c=cloud_cover_df['status_value'], cmap='coolwarm',
                marker='o', alpha=0.7)

    # Establecer etiquetas y marcas del eje y
    plt.yticks([0, 1], ['Despejado', 'Nublado'])

    # Formatear el eje x
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m-%d %H:%M'))
    plt.gcf().autofmt_xdate()

    # Agregar etiquetas y título
    plt.xlabel('Fecha y Hora')
    plt.ylabel('Estado de Nube')
    plt.title('Línea de Tiempo de Cobertura de Nubes')

    # Agregar cuadrícula
    plt.grid(True, alpha=0.3)

    # Guardar la figura
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()

    print(f"Gráfico de línea de tiempo guardado en {output_file}")

def compare_with_table(daily_results_df, table_data):
    """Comparar resultados de análisis con los datos de la tabla proporcionada"""
    comparison_results = []

    for _, row in daily_results_df.iterrows():
        date = row['date']
        date_str = date.strftime('%d/%m/%Y')

        # Encontrar entrada correspondiente en table_data
        table_entry = next((item for item in table_data if item['date'] == date_str), None)

        if table_entry is None:
            print(f"No se encontró entrada correspondiente en la tabla para la fecha {date_str}")
            continue

        # Comparar estado nublado
        analysis_cloudy = row['is_cloudy']
        table_cloudy = table_entry['cloudy'] == 'Yes'

        match = analysis_cloudy == table_cloudy

        comparison_results.append({
            'date': date_str,
            'analysis_cloudy': 'Yes' if analysis_cloudy else 'No',
            'table_cloudy': table_entry['cloudy'],
            'match': match,
            'cloudy_percentage': row['cloudy_percentage'],
            'clear_percentage': row['clear_percentage'],
            'alternating_percentage': row['alternating_percentage']
        })

    return pd.DataFrame(comparison_results)

def main():
    """Función principal de ejecución"""
    if not HDA_AVAILABLE: print("Error: Cliente WEkEO HDA no instalado."); return
    if not CFGRIB_AVAILABLE: print("Advertencia: cfgrib no instalado. El procesamiento GRIB puede fallar.");

    # Credenciales WEkEO
    username = 'chinso' # Replace with your username
    password = 'WhctKLbfHpRVt' # Replace with your password

    # Área de interés (región de EAU)
    bbox_coords = [
        55.43467643191375,
        24.72290881524073,
        55.51147607683789,
        24.78228805026869
    ]

    # Datos de la tabla de la imagen
    table_data = [
        {'date': '01/02/2025', 'cloudy': 'No'},
        {'date': '02/02/2025', 'cloudy': 'Yes'},
        {'date': '03/02/2025', 'cloudy': 'No'},
        {'date': '04/02/2025', 'cloudy': 'Yes'},
        {'date': '05/02/2025', 'cloudy': 'Yes'},
        {'date': '06/02/2025', 'cloudy': 'Yes'}
    ]

    # Crear directorio de resultados
    results_dir = "cloud_cover_results"
    os.makedirs(results_dir, exist_ok=True)

    # Procesar cada día
    all_days_data = pd.DataFrame()
    daily_results = []

    for entry in table_data:
        date_str = entry['date']
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')

        # Establecer rango de tiempo para el día completo
        start_time_str = f"{date_obj.strftime('%Y-%m-%d')} 00:00:00"
        end_time_str = f"{date_obj.strftime('%Y-%m-%d')} 23:59:59"

        print(f"\n{'='*50}")
        print(f"Procesando fecha: {date_str}")
        print(f"{'='*50}")

        try:
            # Obtener datos de cobertura de nubes para este día
            day_download_folder = os.path.join(results_dir, f"data_{date_obj.strftime('%Y%m%d')}")
            cloud_cover_df = get_cloud_cover(
                bbox_coords,
                start_time_str,
                end_time_str,
                username,
                password,
                download_folder=day_download_folder
            )

            if not cloud_cover_df.empty:
                # Guardar datos brutos
                csv_file = os.path.join(results_dir, f"cloud_data_{date_obj.strftime('%Y%m%d')}.csv")
                cloud_cover_df.to_csv(csv_file, index=False)
                print(f"Datos brutos guardados en {csv_file}")

                # Agregar a todos los datos
                all_days_data = pd.concat([all_days_data, cloud_cover_df])

                # Crear gráfico de línea de tiempo
                plot_file = os.path.join(results_dir, f"timeline_{date_obj.strftime('%Y%m%d')}.png")
                plot_cloud_cover_timeline(cloud_cover_df, plot_file)
            else:
                print(f"No se recuperaron datos para {date_str}")

        except Exception as e:
            print(f"Error al procesar {date_str}: {str(e)}")

    if not all_days_data.empty:
        # Guardar datos combinados
        all_days_csv = os.path.join(results_dir, "all_cloud_data.csv")
        all_days_data.to_csv(all_days_csv, index=False)
        print(f"\nDatos combinados guardados en {all_days_csv}")

        # Analizar datos por día
        daily_analysis = analyze_cloud_cover_by_day(all_days_data, bbox_coords)

        if not daily_analysis.empty:
            # Guardar análisis diario
            daily_analysis_csv = os.path.join(results_dir, "daily_analysis.csv")
            daily_analysis.to_csv(daily_analysis_csv, index=False)
            print(f"Análisis diario guardado en {daily_analysis_csv}")

            # Comparar con tabla
            comparison = compare_with_table(daily_analysis, table_data)
            comparison_csv = os.path.join(results_dir, "table_comparison.csv")
            comparison.to_csv(comparison_csv, index=False)
            print(f"Comparación con tabla guardada en {comparison_csv}")

            # Imprimir resultados de comparación
            print("\nComparación con tabla proporcionada:")
            print("=" * 80)
            print(comparison.to_string(index=False))
            print("=" * 80)

            # Calcular porcentaje de coincidencia general
            match_percentage = (comparison['match'].sum() / len(comparison)) * 100
            print(f"\nPorcentaje de coincidencia general: {match_percentage:.2f}%")
        else:
            print("El análisis diario no produjo resultados.")
    else:
        print("\nNo se recuperaron datos para ninguna de las fechas.")

if __name__ == "__main__":
    main()


In [ ]:
"""
Script para extraer datos de cobertura de nubes de EUMETSAT MSG CLM-IODC usando EUMDAC
Autor: Manus
Fecha: Mayo 2025
"""

import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import os
import time
import glob
import sys
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# Importar cliente EUMDAC
try:
    from eumdac import DataStore
    EUMDAC_AVAILABLE = True
    print("Cliente EUMDAC importado correctamente")
except ImportError:
    print("EUMDAC no está instalado. Instálalo con: pip install eumdac")
    EUMDAC_AVAILABLE = False
    sys.exit(1)

# Importar cfgrib para manejo de archivos GRIB
try:
    import cfgrib
    CFGRIB_AVAILABLE = True
    print("cfgrib importado correctamente")
except ImportError:
    print("cfgrib no está instalado. Instálalo con: pip install cfgrib")
    CFGRIB_AVAILABLE = False
    sys.exit(1)

def create_download_folder(base_folder="eumetsat_downloads"):
    """Crear y devolver la ruta a la carpeta de descarga"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    download_folder = os.path.join(base_folder, f"download_{timestamp}")
    os.makedirs(download_folder, exist_ok=True)
    print(f"Carpeta de descarga creada: {download_folder}")
    return download_folder

def setup_eumdac_client(consumer_key, consumer_secret):
    """Inicializar y devolver el cliente EUMDAC con credenciales"""
    try:
        # Inicializar el cliente DataStore con las credenciales
        datastore = DataStore(consumer_key, consumer_secret)
        print("Cliente EUMDAC inicializado correctamente")
        return datastore
    except Exception as e:
        print(f"Error al inicializar el cliente EUMDAC: {str(e)}")
        raise

def search_msg_clm_data(datastore, bbox_coords, start_time, end_time):
    """Buscar datos MSG CLM-IODC con los parámetros especificados"""
    try:
        # Convertir bbox a formato requerido por EUMDAC
        bbox_str = f"{bbox_coords[0]},{bbox_coords[1]},{bbox_coords[2]},{bbox_coords[3]}"

        # Crear filtros de búsqueda
        filters = {
            'dtstart': start_time.isoformat() + 'Z',
            'dtend': end_time.isoformat() + 'Z',
            'bbox': bbox_str,
            'format': 'grib'  # Ajustar según el tipo de archivo disponible
        }

        # Buscar colecciones relacionadas con MSG CLM-IODC
        collections = datastore.get_collections()
        msg_collections = [c for c in collections if 'MSG' in c.name and 'CLM' in c.name]

        if not msg_collections:
            print("No se encontraron colecciones MSG CLM-IODC")
            # Buscar alternativas
            print("Buscando colecciones alternativas relacionadas con nubes...")
            cloud_collections = [c for c in collections if 'CLOUD' in c.name.upper() or 'CLM' in c.name]
            if cloud_collections:
                print(f"Colecciones alternativas encontradas: {[c.name for c in cloud_collections]}")
                msg_collections = cloud_collections
            else:
                print("No se encontraron colecciones alternativas")
                # Mostrar todas las colecciones disponibles para referencia
                print("Colecciones disponibles:")
                for c in collections:
                    print(f" - {c.name}")
                return []

        print(f"Colecciones encontradas: {[c.name for c in msg_collections]}")

        # Buscar productos en las colecciones encontradas
        all_products = []
        for collection in msg_collections:
            print(f"Buscando productos en la colección: {collection.name}")
            products = collection.search_products(filters)
            print(f"Encontrados {len(products)} productos en {collection.name}")
            all_products.extend(products)

        return all_products

    except Exception as e:
        print(f"Error durante la búsqueda: {str(e)}")
        raise

def download_products(products, download_folder, limit=None):
    """Descargar productos y devolver la lista de archivos descargados"""
    if not products:
        print("No hay productos para descargar")
        return []

    # Aplicar límite si se especifica
    if limit is not None and len(products) > limit:
        print(f"Limitando la descarga de {len(products)} a {limit} archivos")
        products_to_download = products[:limit]
    else:
        products_to_download = products

    print(f"Iniciando descarga de {len(products_to_download)} archivos")
    print(f"Los archivos se guardarán en: {download_folder}")

    downloaded_files = []
    for i, product in enumerate(products_to_download):
        try:
            print(f"Descargando archivo {i+1}/{len(products_to_download)}...")
            # Obtener el nombre del archivo
            filename = product.name if hasattr(product, 'name') else f"product_{i+1}.grib"
            file_path = os.path.join(download_folder, filename)

            # Descargar el producto
            product.download(file_path)
            downloaded_files.append(file_path)
            print(f"  ...Descargado {filename}")

        except Exception as e:
            print(f"Error al descargar el archivo {i+1}: {str(e)}")
            continue

    print(f"\nDescarga completa. {len(downloaded_files)} archivos descargados.")
    return downloaded_files

def process_file(file_path, latitude, longitude):
    """Extraer el estado de las nubes de un archivo NetCDF o GRIB"""
    try:
        print(f"Procesando archivo: {os.path.basename(file_path)}")
        ds = None
        if file_path.lower().endswith(('.grib', '.grb')):
            try:
                ds = xr.open_dataset(file_path, engine='cfgrib',
                                     backend_kwargs={'filter_by_keys': {'discipline': 3, 'parameterCategory': 0, 'parameterNumber': 7},
                                                     'indexpath': ''})
            except Exception as e:
                print(f"Error al abrir el archivo GRIB {os.path.basename(file_path)} con claves: {str(e)}. Intentando sin filtros...")
                try: ds = xr.open_dataset(file_path, engine='cfgrib', backend_kwargs={'indexpath': ''})
                except Exception as e2: print(f"Error en la apertura alternativa de GRIB: {str(e2)}"); return None
        elif file_path.lower().endswith(('.nc', '.netcdf')):
            ds = xr.open_dataset(file_path)
        else: print(f"Tipo de archivo no soportado: {os.path.basename(file_path)}"); return None

        if ds is None: return None

        with ds:
            # Imprimir estructura del dataset para depuración
            print(f"Estructura del dataset:\n{ds}")

            # Encontrar la variable de tiempo
            time_var = next((v for v in ds.variables if v in ds.coords or v in ds.variables), None)
            if not time_var: print(f"No se encontró variable de tiempo en {os.path.basename(file_path)}"); return None
            timestamp = pd.to_datetime(ds[time_var].values.item() if hasattr(ds[time_var].values, 'item') else ds[time_var].values).to_pydatetime()

            # Encontrar la variable de máscara de nubes
            cloud_var_name = next((v for v in ['cloud_mask', 'p260537', 'CLM'] if v in ds.data_vars), None)
            if not cloud_var_name:
                print(f"No se encontró variable de nubes en {os.path.basename(file_path)}")
                # Mostrar todas las variables disponibles
                print(f"Variables disponibles: {list(ds.data_vars.keys())}")
                return None

            cloud_mask = ds[cloud_var_name]

            # Extraer el valor de nube para la ubicación especificada
            if 'latitude' in ds.coords and 'longitude' in ds.coords:
                try:
                    if 'values' in cloud_mask.dims:
                        lats, lons = cloud_mask.latitude.values, cloud_mask.longitude.values
                        distances = np.sqrt((lats - latitude)**2 + (lons - longitude)**2)
                        cloud_value = cloud_mask.isel(values=np.argmin(distances)).item()
                    else:
                        cloud_value = cloud_mask.sel(latitude=latitude, longitude=longitude, method='nearest').item()
                except Exception as e: print(f"Error al extraer el valor de nube: {str(e)}"); return None
            else: print(f"No se encontraron coordenadas lat/lon en {os.path.basename(file_path)}"); return None

            # Mapear valores a estados de nube
            status_map = {0: 'clear (water)', 1: 'clear (land)', 2: 'cloudy', 3: 'not processed'}
            status = status_map.get(cloud_value, f'unknown ({cloud_value})')
            clear_status = 'clear' if status.startswith('clear') else status

            result = {'datetime': timestamp, 'cloud_status': clear_status, 'cloud_value': cloud_value, 'file': os.path.basename(file_path), 'file_path': file_path}
            print(f"Procesado {os.path.basename(file_path)}: {status} (valor: {cloud_value})")
            return result
    except Exception as e:
        print(f"Error grave al procesar {os.path.basename(file_path)}: {str(e)}")
        return None

def calculate_daylight_hours(latitude, longitude, date):
    """Calcular horas aproximadas de amanecer y atardecer para una ubicación y fecha dadas"""
    # Cálculo simplificado para demostración
    # Para cálculos más precisos, considerar bibliotecas como astral o ephem

    # Convertir fecha a día del año
    day_of_year = date.timetuple().tm_yday

    # Calcular ángulo de declinación
    declination = 23.45 * np.sin(np.radians(360/365 * (day_of_year - 81)))

    # Calcular duración del día en horas
    day_length = 24 - (24/np.pi) * np.arccos(
        (np.sin(np.radians(-0.83)) + np.sin(np.radians(latitude)) *
         np.sin(np.radians(declination))) /
        (np.cos(np.radians(latitude)) * np.cos(np.radians(declination)))
    )

    # Calcular horas de amanecer y atardecer
    noon = datetime(date.year, date.month, date.day, 12, 0)
    sunrise = noon - timedelta(hours=day_length/2)
    sunset = noon + timedelta(hours=day_length/2)

    return sunrise, sunset

def analyze_cloud_cover_by_day(cloud_cover_df, bbox_coords):
    """Analizar datos de cobertura de nubes por día, enfocándose en períodos de 15 minutos durante el día"""
    if cloud_cover_df.empty:
        return pd.DataFrame()

    # Calcular centro del bbox para cálculos de luz diurna
    latitude = (bbox_coords[1] + bbox_coords[3]) / 2
    longitude = (bbox_coords[0] + bbox_coords[2]) / 2

    # Agrupar datos por fecha
    cloud_cover_df['date'] = cloud_cover_df['datetime'].dt.date

    # Inicializar resultados
    daily_results = []

    # Procesar cada día
    for date, group in cloud_cover_df.groupby('date'):
        date_obj = datetime.combine(date, datetime.min.time())

        # Calcular amanecer y atardecer para esta fecha
        sunrise, sunset = calculate_daylight_hours(latitude, longitude, date_obj)

        # Filtrar datos para horas diurnas
        daylight_data = group[(group['datetime'] >= sunrise) & (group['datetime'] <= sunset)]

        if daylight_data.empty:
            print(f"No hay datos diurnos disponibles para {date}")
            continue

        # Remuestrear a intervalos de 15 minutos
        # Primero, establecer datetime como índice
        daylight_data = daylight_data.set_index('datetime')

        # Crear bins de 15 minutos desde el amanecer hasta el atardecer
        time_bins = pd.date_range(start=sunrise, end=sunset, freq='15min')

        # Inicializar contadores
        total_periods = len(time_bins) - 1
        cloudy_periods = 0
        clear_periods = 0
        alternating_count = 0
        previous_status = None

        # Analizar cada período de 15 minutos
        for i in range(len(time_bins) - 1):
            period_start = time_bins[i]
            period_end = time_bins[i+1]

            # Obtener puntos de datos en este período
            period_data = daylight_data[(daylight_data.index >= period_start) &
                                        (daylight_data.index < period_end)]

            if period_data.empty:
                # No hay datos para este período
                continue

            # Determinar el estado dominante en este período
            status_counts = period_data['cloud_status'].value_counts()
            if status_counts.empty:
                continue

            current_status = status_counts.index[0]

            # Contar períodos nublados y despejados
            if current_status == 'cloudy':
                cloudy_periods += 1
            elif current_status == 'clear':
                clear_periods += 1

            # Verificar patrón alternante
            if previous_status is not None and current_status != previous_status:
                alternating_count += 1

            previous_status = current_status

        # Calcular porcentajes
        if total_periods > 0:
            cloudy_percentage = (cloudy_periods / total_periods) * 100
            clear_percentage = (clear_periods / total_periods) * 100
            alternating_percentage = (alternating_count / (total_periods - 1)) * 100 if total_periods > 1 else 0
        else:
            cloudy_percentage = clear_percentage = alternating_percentage = 0

        # Determinar clasificación diaria basada en predominancia
        is_cloudy = cloudy_percentage > clear_percentage

        # Agregar a resultados
        daily_results.append({
            'date': date,
            'sunrise': sunrise,
            'sunset': sunset,
            'total_15min_periods': total_periods,
            'cloudy_periods': cloudy_periods,
            'clear_periods': clear_periods,
            'alternating_count': alternating_count,
            'cloudy_percentage': cloudy_percentage,
            'clear_percentage': clear_percentage,
            'alternating_percentage': alternating_percentage,
            'is_cloudy': is_cloudy
        })

    return pd.DataFrame(daily_results)

def plot_cloud_cover_timeline(cloud_cover_df, output_file):
    """Crear una visualización de línea de tiempo de datos de cobertura de nubes"""
    if cloud_cover_df.empty:
        print("No hay datos para graficar")
        return

    # Crear una nueva figura
    plt.figure(figsize=(12, 6))

    # Mapear estado de nube a valores numéricos para graficar
    status_map = {'clear': 0, 'cloudy': 1, 'not processed': -1}
    cloud_cover_df['status_value'] = cloud_cover_df['cloud_status'].map(lambda x: status_map.get(x, -1))

    # Graficar los datos
    plt.scatter(cloud_cover_df['datetime'], cloud_cover_df['status_value'],
                c=cloud_cover_df['status_value'], cmap='coolwarm',
                marker='o', alpha=0.7)

    # Establecer etiquetas y marcas del eje y
    plt.yticks([0, 1], ['Despejado', 'Nublado'])

    # Formatear el eje x
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m-%d %H:%M'))
    plt.gcf().autofmt_xdate()

    # Agregar etiquetas y título
    plt.xlabel('Fecha y Hora')
    plt.ylabel('Estado de Nube')
    plt.title('Línea de Tiempo de Cobertura de Nubes')

    # Agregar cuadrícula
    plt.grid(True, alpha=0.3)

    # Guardar la figura
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()

    print(f"Gráfico de línea de tiempo guardado en {output_file}")

def compare_with_table(daily_results_df, table_data):
    """Comparar resultados de análisis con los datos de la tabla proporcionada"""
    comparison_results = []

    for _, row in daily_results_df.iterrows():
        date = row['date']
        date_str = date.strftime('%d/%m/%Y')

        # Encontrar entrada correspondiente en table_data
        table_entry = next((item for item in table_data if item['date'] == date_str), None)

        if table_entry is None:
            print(f"No se encontró entrada correspondiente en la tabla para la fecha {date_str}")
            continue

        # Comparar estado nublado
        analysis_cloudy = row['is_cloudy']
        table_cloudy = table_entry['cloudy'] == 'Yes'

        match = analysis_cloudy == table_cloudy

        comparison_results.append({
            'date': date_str,
            'analysis_cloudy': 'Yes' if analysis_cloudy else 'No',
            'table_cloudy': table_entry['cloudy'],
            'match': match,
            'cloudy_percentage': row['cloudy_percentage'],
            'clear_percentage': row['clear_percentage'],
            'alternating_percentage': row['alternating_percentage']
        })

    return pd.DataFrame(comparison_results)

def main():
    """Función principal de ejecución"""
    if not EUMDAC_AVAILABLE:
        print("Error: EUMDAC no está disponible.")
        return

    # Credenciales EUMDAC (reemplazar con credenciales reales)
    consumer_key = "tu_consumer_key"
    consumer_secret = "tu_consumer_secret"

    # Área de interés (región de EAU)
    bbox_coords = [
        55.43467643191375,
        24.72290881524073,
        55.51147607683789,
        24.78228805026869
    ]

    # Datos de la tabla de la imagen
    table_data = [
        {'date': '01/02/2025', 'cloudy': 'No'},
        {'date': '02/02/2025', 'cloudy': 'Yes'},
        {'date': '03/02/2025', 'cloudy': 'No'},
        {'date': '04/02/2025', 'cloudy': 'Yes'},
        {'date': '05/02/2025', 'cloudy': 'Yes'},
        {'date': '06/02/2025', 'cloudy': 'Yes'}
    ]

    # Crear directorio de resultados
    results_dir = "cloud_cover_results"
    os.makedirs(results_dir, exist_ok=True)

    try:
        print("Conectando con EUMDAC para obtener datos reales...")
        datastore = setup_eumdac_client(consumer_key, consumer_secret)

        # Procesar cada día
        all_days_data = pd.DataFrame()

        for entry in table_data:
            date_str = entry['date']
            date_obj = datetime.strptime(date_str, '%d/%m/%Y')

            # Establecer rango de tiempo para el día completo
            start_time = datetime.combine(date_obj, datetime.min.time())
            end_time = datetime.combine(date_obj, datetime.max.time())

            print(f"\n{'='*50}")
            print(f"Procesando fecha: {date_str}")
            print(f"{'='*50}")

            try:
                # Buscar productos para este día
                products = search_msg_clm_data(datastore, bbox_coords, start_time, end_time)

                if products:
                    # Descargar productos
                    day_download_folder = os.path.join(results_dir, f"data_{date_obj.strftime('%Y%m%d')}")
                    os.makedirs(day_download_folder, exist_ok=True)
                    files = download_products(products, day_download_folder)

                    # Procesar archivos descargados
                    day_results = []
                    for file_path in files:
                        result = process_file(file_path, (bbox_coords[1] + bbox_coords[3])/2, (bbox_coords[0] + bbox_coords[2])/2)
                        if result:
                            day_results.append(result)

                    if day_results:
                        day_df = pd.DataFrame(day_results)

                        # Guardar datos brutos
                        csv_file = os.path.join(results_dir, f"cloud_data_{date_obj.strftime('%Y%m%d')}.csv")
                        day_df.to_csv(csv_file, index=False)
                        print(f"Datos brutos guardados en {csv_file}")

                        # Agregar a todos los datos
                        all_days_data = pd.concat([all_days_data, day_df])

                        # Crear gráfico de línea de tiempo
                        plot_file = os.path.join(results_dir, f"timeline_{date_obj.strftime('%Y%m%d')}.png")
                        plot_cloud_cover_timeline(day_df, plot_file)
                    else:
                        print(f"No se pudieron procesar archivos para {date_str}")
                else:
                    print(f"No se encontraron productos para {date_str}")
            except Exception as e:
                print(f"Error al procesar {date_str}: {str(e)}")

        if not all_days_data.empty:
            # Guardar datos combinados
            all_days_csv = os.path.join(results_dir, "all_cloud_data.csv")
            all_days_data.to_csv(all_days_csv, index=False)
            print(f"\nDatos combinados guardados en {all_days_csv}")

            # Analizar datos por día
            daily_analysis = analyze_cloud_cover_by_day(all_days_data, bbox_coords)

            if not daily_analysis.empty:
                # Guardar análisis diario
                daily_analysis_csv = os.path.join(results_dir, "daily_analysis.csv")
                daily_analysis.to_csv(daily_analysis_csv, index=False)
                print(f"Análisis diario guardado en {daily_analysis_csv}")

                # Comparar con tabla
                comparison = compare_with_table(daily_analysis, table_data)
                comparison_csv = os.path.join(results_dir, "table_comparison.csv")
                comparison.to_csv(comparison_csv, index=False)
                print(f"Comparación con tabla guardada en {comparison_csv}")

                # Imprimir resultados de comparación
                print("\nComparación con tabla proporcionada:")
                print("=" * 80)
                print(comparison.to_string(index=False))
                print("=" * 80)

                # Calcular porcentaje de coincidencia general
                match_percentage = (comparison['match'].sum() / len(comparison)) * 100
                print(f"\nPorcentaje de coincidencia general: {match_percentage:.2f}%")
            else:
                print("El análisis diario no produjo resultados.")
        else:
            print("\nNo se recuperaron datos para ninguna de las fechas.")

    except Exception as e:
        print(f"Error durante la ejecución: {str(e)}")

if __name__ == "__main__":
    main()
